In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
seed = 2024

import warnings
warnings.filterwarnings("ignore")

# ML tools 

import tensorflow as tf
import keras
import keras_nlp
import keras_cv
from keras import ops

keras.utils.set_random_seed(seed)

import cv2
import tensorflow_io as tfio
from kaggle_datasets import KaggleDatasets
import tensorflow_datasets as tfds

from keras import Input, Model
from keras.models import load_model

from keras.layers import Conv2D, DepthwiseConv2D, Dense, Activation, BatchNormalization, LayerNormalization, MultiHeadAttention, Embedding, Subtract, Add, Multiply, GlobalAveragePooling2D, GlobalAveragePooling1D, LayerNormalization
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import *
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tqdm.notebook import tqdm
print(f"Requirements loaded, keras : v{keras.__version__}, Tensorflow : v{tf.__version__}")

2024-05-17 03:32:59.400101: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 03:32:59.400236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 03:32:59.553488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Requirements loaded, keras : v3.2.1, Tensorflow : v2.15.0


# Dataloader setting
- original dataset(input as tf.ds) : image only or image/label paired dataset
- target dataset(output as tf.ds) : (image1, image2), (image1, image2, image3), (image1, image2, label)
    - Also, implement the mixing function : merges 2 homogenous dataset

In [2]:
basic_aug = keras.Sequential([keras.layers.RandomFlip(), keras.layers.RandomRotation(factor = 0.25)])

In [3]:
aug_layers = keras_cv.layers.RandAugment.get_standard_policy(
    value_range=(0, 255), magnitude=0.2, magnitude_stddev=0.15
)
aug_layers = aug_layers + [keras_cv.layers.GridMask()]
aug_layers.pop(0)
print("RandAug Component in this SSL module : ",[layer.name for layer in aug_layers])
randaug = keras_cv.layers.RandomAugmentationPipeline(
    layers=aug_layers, augmentations_per_image=3, seed = seed, auto_vectorize = True
)

RandAug Component in this SSL module :  ['equalization', 'solarization', 'random_color_degeneration', 'random_contrast', 'random_brightness', 'random_shear', 'random_shear_1', 'random_translation', 'random_translation_1', 'grid_mask']


In [4]:
def get_other_augs_(res):

    crop_resize_global = keras.Sequential([keras.layers.RandomCrop(int(0.8*res), int(0.8*res)),
                                        keras.layers.Resizing(res, res)
                                       ])
    crop_resize_local = keras.Sequential([keras.layers.RandomCrop(int(0.5*res), int(0.5*res)),
                                        keras.layers.Resizing(res, res)
                                       ])
    crop_resize_medium = keras.Sequential([keras.layers.RandomCrop(int(0.7*res), int(0.7*res)),
                                        keras.layers.Resizing(res, res)
                                       ])
    return crop_resize_global, crop_resize_medium, crop_resize_local

In [5]:
def get_map_fn(res, input_type = None, output_type = None, n_view = 2):
    # input_type as "supervised", "with_label", "with label" / OR / "unsupervised", "without label", "without_label", "image_only", "image"
    # output_ytpe as "ssl", "ssl_with_label" / if ssl, image output : image1, image2, ..., image_n_view
    assert n_view >= 2, "Augmented View number must be >= 2"
    assert input_type in ["supervised", "with_label", "with label", "unsupervised", "without label", "without_label", "image_only", "image"], 'Pick one of input_type : "supervised", "with_label", "with label",\n "unsupervised", "without label", "without_label", "image_only", "image"'
    assert output_type in ["ssl", "ssl_with_label"], 'Pick one of output_type : "ssl", "ssl_with_label"'
    crop_resize_global, crop_resize_medium, crop_resize_local = get_other_augs_(res)
    def map_fn(input_data):
        if input_type in ["supervised", "with_label", "with label"]:
            image, label = input_data
        elif input_type in ["unsupervised", "without label", "without_label", "image_only", "image"]:
            image = input_data
        batch_size = ops.shape(image)[0]
        
        aug_ = basic_aug(image)
        aug_ = randaug(aug_)
        
        global_image = crop_resize_global(aug_)
        
        if n_view == 2:
            if output_type == "ssl":
                return (image, global_image)
            elif output_type == "ssl_with_label":
                return (image, global_image, label)
        elif n_view == 3:
            medium_image = crop_resize_medium(aug_)
            if output_type == "ssl":
                return (image, global_image, medium_image)
            elif output_type == "ssl_with_label":
                return (image, global_image, medium_image, label)
        elif n_view > 3:
            medium_image = crop_resize_medium(aug_)
            local_images = [crop_resize_local(image) for _ in range(n_view - 3)]
            local_images = tuple(local_images)
            outputs = (image, global_image, medium_image) + local_images
            if output_type == "ssl":
                return outputs
            elif output_type == "ssl_with_label":
                return outputs+(label)
    return map_fn


# Helper functions

In [6]:
def get_flops(model, model_inputs) -> float:
        """
        Calculate FLOPS [GFLOPs] for a tf.keras.Model or tf.keras.Sequential model
        in inference mode. It uses tf.compat.v1.profiler under the hood.
        Code reference : https://github.com/tensorflow/tensorflow/issues/32809
        """
        # if not hasattr(model, "model"):
        #     raise wandb.Error("self.model must be set before using this method.")
        
        if not isinstance(
            model, (tf.keras.models.Sequential, tf.keras.models.Model, keras.models.Model, keras.Sequential, keras.Model)
        ):
            raise ValueError(
                "Calculating FLOPS is only supported for "
                "`tf.keras.Model` and `tf.keras.Sequential` instances."
            )

        from tensorflow.python.framework.convert_to_constants import (
            convert_variables_to_constants_v2_as_graph,
        )

        # Compute FLOPs for one sample
        batch_size = 1
        inputs = [
            tf.TensorSpec([batch_size] + inp.shape[1:], inp.dtype)
            for inp in model_inputs
        ]

        # convert tf.keras model into frozen graph to count FLOPs about operations used at inference
        real_model = tf.function(model).get_concrete_function(inputs)
        frozen_func, _ = convert_variables_to_constants_v2_as_graph(real_model)

        # Calculate FLOPs with tf.profiler
        run_meta = tf.compat.v1.RunMetadata()
        opts = (
            tf.compat.v1.profiler.ProfileOptionBuilder(
                tf.compat.v1.profiler.ProfileOptionBuilder().float_operation()
            )
            .with_empty_output()
            .build()
        )

        flops = tf.compat.v1.profiler.profile(
            graph=frozen_func.graph, run_meta=run_meta, cmd="scope", options=opts
        )

        tf.compat.v1.reset_default_graph()

        # convert to GFLOPs
        return (flops.total_float_ops / 1e9)

In [7]:
class AttentionPooling(keras.layers.Layer):
    def __init__(self, attention_heads, attention_dims = None, bias = False, scale = None, 
                 dropout_rate = 0.05, **kwargs):
        super().__init__(**kwargs)
        self.n_heads = attention_heads
        self.n_dims = attention_dims
        self.bias = bias
        self.scale = scale
        self.dropout_rate = dropout_rate
        
    def build(self, input_shape):
        # query, key
        query_dims = input_shape[0][-1]
        key_length = input_shape[1][1]
        
        if self.n_dims == None:
            embed_dims = query_dims
        else:
            embed_dims = self.n_dims
        self.embed_dims = embed_dims
        self.per_head_dims = embed_dims//self.n_heads
        
        self.scale = self.scale if self.scale != None else embed_dims**-0.5
        
        self.query_embed_fn = keras.layers.Dense(units = embed_dims, use_bias = self.bias,
                                               name = "Q_Embedding_Dense_layer")
        self.key_embed_fn = keras.layers.Dense(units = embed_dims, use_bias = self.bias,
                                               name = "K_Embedding_Dense_layer")
        self.value_embed_fn = keras.layers.Dense(units = embed_dims, use_bias = self.bias,
                                               name = "V_Embedding_Dense_layer")
        
        self.softmax = keras.layers.Activation("softmax", name = "AttentionWeightSoftmax")
        self.proj = keras.layers.Dense(units = query_dims, use_bias = self.bias, 
                                      name = "ProjectToOriginalDimension")
        self.att_dropout = keras.layers.Dropout(self.dropout_rate)
        self.proj_dropout = keras.layers.Dropout(self.dropout_rate)
        super().build(input_shape)
        
    def call(self, inputs, **kwargs):
        if len(inputs) == 2:
            q, k = inputs
            value_ = False
        elif len(inputs) == 3:
            q, k, v = inputs
            value_ = True
            
        if len(ops.shape(q)) == 2:
            q = q[:, tf.newaxis, :]
        if len(ops.shape(k)) == 4:
            b_, w_, h_, dims_ = ops.shape(k)
            k = ops.reshape(k, [b_, w_*h_, dims_])
        if (value_) and (len(ops.shape(v))) == 4:
            b_, w_, h_, dims_ = ops.shape(v)
            v = ops.reshape(v, [b_, w_*h_, dims_])
            
        batch_size, query_length, q_dims = ops.shape(q)
        _, key_length, k_dms = ops.shape(k)
        
        query = self.query_embed_fn(q) * self.scale #batch, 1(or, query length), q_dims
        query = ops.reshape(query, [batch_size, query_length, self.n_heads, self.per_head_dims])
        
        key = self.key_embed_fn(k)
        key = ops.reshape(key, [batch_size, key_length, self.n_heads, self.per_head_dims])
        
        if value_:
            value = self.value_embed_fn(v)#각각 batch, token_length, heads, per_head_dims
        else:
            value = self.value_embed_fn(k)
        value = ops.reshape(value, [batch_size, key_length, self.n_heads, self.per_head_dims])
        attention_score = keras.ops.einsum("abhd, achd -> ahbc",
                                          query, key) #b = query length, c = key length
        attention_weight = self.softmax(attention_score)
        attention_weight = self.att_dropout(attention_weight)
        self.attention_weight = attention_weight 
        attended_output = keras.ops.einsum("ahbc, achd -> abhd",
                                          attention_weight, value)
        attended_output = keras.ops.reshape(attended_output, 
                                           [batch_size, query_length, self.n_heads*self.per_head_dims]
                                           )
        attended_output = self.proj(attended_output)
        attended_output = self.proj_dropout(attended_output)
        return attended_output, attention_weight

In [8]:
class PatchEncoder(keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = keras.layers.Dense(units=projection_dim)
        self.position_embedding = keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = ops.expand_dims(
            ops.arange(start=0, stop=self.num_patches, step=1), axis=0
        )
        projected_patches = self.projection(patch)
        encoded = projected_patches + self.position_embedding(positions)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches})
        return config

# General Context Vision Transformer implementation
- Reference : [another kaggle notebook, identical implementation](https://www.kaggle.com/code/hskimjjys/general-context-vit-script/).

In [9]:
class SE(keras.layers.Layer):
    def __init__(self, output_dim = None, squeeze_rate = 0.25, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.rate = squeeze_rate
    def build(self, input_shape) : #batch_size, h, w, dims
        if self.output_dim == None:
            self.output_dim = input_shape[-1]
        else:
            pass
        self.avg_pool = keras.layers.GlobalAveragePooling2D(keepdims = True, name = "AvgPooling")
        self.mlps = keras.Sequential([keras.layers.Dense(units = int(self.rate * self.output_dim),
                                                        use_bias = False, name = "Dense1"),
                                      keras.layers.Activation("gelu", name = "GeluAct"),
                                      keras.layers.Dense(units = self.output_dim, use_bias = False, name = "Dense2"),
                                      keras.layers.Activation("sigmoid", name = "Excitation_Sigmoid")
                                     ])
        #super().build(input_shape)
    def call(self, inputs, **kwargs):
        pooled = self.avg_pool(inputs)
        weights = self.mlps(pooled)
        return inputs * weights
    
class DownSampler(keras.layers.Layer):
    def __init__(self, keepdims = False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims
    def build(self, input_shape):
        embed_dims = input_shape[-1]
        out_dim = embed_dims if self.keepdims else 2*embed_dims
        self.fused_mbconv = keras.Sequential([keras.layers.DepthwiseConv2D(kernel_size = 3, padding = 'same', use_bias = False, name = "DWConv"),
                                             keras.layers.Activation("gelu", name = 'GeluAct'),
                                             SE(name = "SqueezeAndExcitation2D"),
                                             keras.layers.Conv2D(filters = embed_dims, kernel_size = 1, padding = 'same', use_bias = False, name = "PointWiseConv")],
                                            name = "Fused_MBConvLayer")
        self.down_conv = keras.layers.Conv2D(filters = out_dim, kernel_size = 3, strides = 2, padding = 'same', use_bias = False, name = "DownConvolution")
        self.layernorm1 = keras.layers.LayerNormalization(epsilon = 1e-5, name = 'LayerNorm1')
        self.layernorm2 = keras.layers.LayerNormalization(epsilon = 1e-5, name = 'LayerNorm2')
    def call(self, inputs, **kwargs):
        x = self.layernorm1(inputs)
        x += self.fused_mbconv(inputs)
        x = self.down_conv(x)
        return self.layernorm2(x)
    
class MLP(keras.layers.Layer):
    def __init__(self, middle_dim = None, output_dim = None,
                activation = 'gelu', dropout = 0.2, **kwargs):
        super().__init__(**kwargs)
        self.middle_dim = middle_dim
        self.output_dim = output_dim
        self.activation = activation
        self.dropout_rate = dropout
    def build(self, input_shape):
        self.input_dims = input_shape[-1]
        self.middle_dim = int(1.5*self.input_dims) if self.middle_dim == None else self.middle_dim
        self.output_dim = self.input_dims if self.output_dim == None else self.output_dim
        self.mlp1 = keras.layers.Dense(units = self.middle_dim, name = "FirstMLP")
        self.act = keras.layers.Activation(self.activation, name = "MiddleActivation")
        self.mlp2 = keras.layers.Dense(units = self.output_dim, name = "SecondMLP")
        self.drop1 = keras.layers.Dropout(self.dropout_rate, name = "Dropout1")
        self.drop2 = keras.layers.Dropout(self.dropout_rate, name = "Dropout2")
    def call(self, inputs, **kwargs):
        x = self.mlp1(inputs)
        x = self.act(x)
        x = self.drop1(x)
        x = self.mlp2(x)
        x = self.drop2(x)
        return x
    
class PatchEmbedding(keras.layers.Layer):
    def __init__(self, embed_dim, patching_type = "conv", #conv or tokenlearner
                 **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.patching_type = patching_type
    def build(self, input_shape):
        if (self.patching_type == "tokenlearner") or (self.patching_type == "token_learner"):
            self.proj = keras.layers.Conv2D(self.embed_dim, kernel_size = 3, strides = 2, padding = 'same', name = "projection_conv") #Overlapping patches
            #token learner implementation
            batch_size, w, h, filters = input_shape
            n_tokens = (w//4) * (h//4) ; self.resized_w, self.resized_h = int(w//4), int(h/4)
            self.input_seq_flatten = keras.layers.Reshape([1, -1, self.embed_dim], name = "image_to_sequence_reshape")
            self.layer_norm = keras.layers.LayerNormalization(epsilon = 1e-5)
            self.attention_ops = keras.Sequential([keras.layers.Conv2D(n_tokens, kernel_size = 3, activation = "gelu", use_bias = False, padding = 'same'),
                                                  keras.layers.Conv2D(n_tokens, kernel_size = 3, activation = "gelu", use_bias = False, padding = 'same'),
                                                  keras.layers.Conv2D(n_tokens, kernel_size = 3, activation = "sigmoid", use_bias = False, padding = 'same'),
                                                  keras.layers.Reshape([-1, n_tokens]), #batch_size, HW, n_tokens
                                                  keras.layers.Permute([2,1])], #batch_size, n_tokens, HW
                                                 name = "Conv_for_attention_weight")
        else:
            self.proj = keras.layers.Conv2D(self.embed_dim, kernel_size = 3, strides = 2, padding = 'same') #Overlapping patches
            self.down_sample = DownSampler(keepdims = True, name = "DownSampler_after_projection")
    def call(self, inputs, **kwargs):
        if (self.patching_type == "tokenlearner") or (self.patching_type == "token_learner"):
            #token learner implementation
            norm_input = self.layer_norm(inputs)
            proj_inputs = self.proj(norm_input)
            seq_inputs = self.input_seq_flatten(proj_inputs) #batch, 1, HW, embed_dims
            att_weights = self.attention_ops(proj_inputs) #batch, n_tokens, HW
            att_weights = ops.expand_dims(att_weights, axis = -1) #batch, n_tokens, HW, 1
            attended = att_weights * seq_inputs #batch, n_tokens, HW, embed_dims
            attended = ops.mean(attended, axis = 2) #batch, n_tokens, embed_dims
            #reshape to 2D array
            attended = ops.reshape(attended, [-1, self.resized_w, self.resized_h, self.embed_dim]
                                  )
            return attended
        else:
            x = self.proj(inputs)
            x = self.down_sample(x)
            return x
        
class FeatExtract(keras.layers.Layer):
    def __init__(self, keepdims = False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims
    def build(self, input_shape):
        batch_size, H, W, embed_dims = input_shape
        self.fused_mbconv = keras.Sequential([keras.layers.DepthwiseConv2D(kernel_size = 3, padding = 'same', use_bias = False, name = "DWConv"),
                                             keras.layers.Activation("gelu", name = 'GeluAct'),
                                             SE(name = "SqueezeAndExcitation2D"),
                                             keras.layers.Conv2D(filters = embed_dims, kernel_size = 1, padding = 'same', use_bias = False, name = "PointWiseConv")],
                                            name = "Fused_MBConvLayer")
        if self.keepdims == False:
            self.pool = keras.layers.MaxPooling2D(name = "FeatExtractMaxPool2D")
    def call(self, inputs):
        x = inputs + self.fused_mbconv(inputs)
        if self.keepdims == False:
            return self.pool(x)
        return x
    
class GlobalQueryGenerator(keras.layers.Layer):
    def __init__(self, keepdims = False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims #Keepdims는 여기서 0과 1로 이루어진 list도 될 수 있다 -> FeatExtract layer를 keepdims의 원소 갯수만큼 repeat!
    def build(self, input_shape):
        self.q_generator = keras.Sequential([FeatExtract(keepdims = keepdim, name = f"FeatureExtraction_{idx+1}") for idx, keepdim in enumerate(self.keepdims)])
    def call(self, inputs):
        return self.q_generator(inputs)
    
class WindowAttention(keras.layers.Layer):
    def __init__(self, window_size, n_heads, global_query, #제공된다면 global, 아니라면 local mHSA -> 0 or 1
                qkv_bias = True, qk_scale = None,
                dropout_rate = 0.05, return_attention_weights = False, **kwargs):
        super().__init__(**kwargs)
        self.window_size = (window_size, window_size)
        self.n_heads = n_heads
        self.global_query = global_query
        self.bias = qkv_bias
        self.scale = qk_scale
        self.dropout_rate = dropout_rate
        self.return_attention_weights = return_attention_weights
    def build(self, input_shape) :
        #input = [query, key, value]
        embed_dims = input_shape[0][-1]
        head_dims = embed_dims//self.n_heads
        self.scale = self.scale if self.scale != None else embed_dims**-0.5
        self.qkv_size = 3-int(self.global_query)
        self.qkv_embed_fn = keras.layers.Dense(units = embed_dims * self.qkv_size, use_bias = self.bias,
                                 name = "QKV_Embedding_Dense_layer")
        self.softmax = keras.layers.Activation("softmax", name = "AttentionWeightSoftmax") #for attention weight computation
        self.proj = keras.layers.Dense(units = embed_dims, use_bias = self.bias, name = "Projection")
        self.attention_dropout = keras.layers.Dropout(self.dropout_rate)
        self.projection_dropout = keras.layers.Dropout(self.dropout_rate)
        self.relative_position_bias_table = self.add_weight(
            name="relative_position_bias_table",
            shape=[
                (2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1),
                self.n_heads,
            ],
            initializer=keras.initializers.TruncatedNormal(stddev=0.02),
            trainable=True,
            dtype=self.dtype,
        ) #<- learnable weight of relational position. 위 window size = 4의 예시에서, 임의의 두 지점 간 거리의 경우의 수는 총 49개 -> 이에 해당하는 weight tensor를 만듬.
        super().build(input_shape)
    def get_relative_position_index(self): #<- window 내 2 지점 간 거리의 index matrix.
        coords_h = ops.arange(self.window_size[0])
        coords_w = ops.arange(self.window_size[1])
        coords = ops.stack(ops.meshgrid(coords_h, coords_w, indexing="ij"), axis=0)
        coords_flatten = ops.reshape(coords, [2, -1])
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = ops.transpose(relative_coords, axes=[1, 2, 0])
        relative_coords_xx = relative_coords[:, :, 0] + self.window_size[0] - 1
        relative_coords_yy = relative_coords[:, :, 1] + self.window_size[1] - 1
        relative_coords_xx = relative_coords_xx * (2 * self.window_size[1] - 1)
        relative_position_index = relative_coords_xx + relative_coords_yy
        return relative_position_index
    def call(self, inputs, **kwargs):
        #input : key(=value), global query OR key only
        # input component shape : batch*n_windows, h_window*w_window, embed_dims -> level/block 설계 시 repeat 처리 후 attention에 feed
        if self.global_query :
            inputs, q_global = inputs
            batch_size = ops.shape(q_global)[0]
        else:
            inputs = inputs[0]
        batch_, token_length, embed_dims = ops.shape(inputs) #global query는 query generator에 의해 token_length 개 만큼의 token으로 전체 이미지/feature map을 압축한 상태
        
        qkv = self.qkv_embed_fn(inputs) #batch*n_windows, h_w * w_w, qkv_size * embed_dims
        
        qkv = ops.reshape(qkv, [-1, token_length, self.qkv_size, self.n_heads, embed_dims//self.n_heads])
        qkv = ops.transpose(qkv, [2, 0, 3, 1, 4]) #qkv_size, batch_, n_heads, token_length, C
        
        #QKV 분리
        if self.global_query:
            k, v = ops.split(qkv, 2, axis = 0) #각각 batch_, n_heads, token_length, C
            #repeat the global query tensor
            # batch_size, n_query_tokens, dims -> batch_(=batch * n_windows), n_query_tokens, dims
            q_global = ops.repeat(q_global, batch_//batch_size, axis = 0) #->batch_, n_query_tokens, dims
            q = ops.reshape(q_global, [batch_, token_length, self.n_heads, embed_dims//self.n_heads])
            q = ops.transpose(q, [0, 2, 1, 3]
                             )
        else:
            q, k, v = ops.split(qkv, 3, axis = 0)
            q = ops.squeeze(q, axis = 0)
        k = ops.squeeze(k, axis = 0)
        v = ops.squeeze(v, axis = 0)
        
        q *= self.scale #batch_, n_heads, token_length, dimension_per_heads(=C)
        attention_score = q@ops.transpose(k, [0, 1, 3, 2]) #batch_, n_heads, token_length, token_length
        
        #positional encoding(bias) 계산 -> attention score에 더해 주기
        # Code from original keras homepage
        relative_position_bias = ops.take(
            self.relative_position_bias_table,
            ops.reshape(self.get_relative_position_index(), [-1]),
        )
        relative_position_bias = ops.reshape(
            relative_position_bias,
            [
                self.window_size[0] * self.window_size[1],
                self.window_size[0] * self.window_size[1],
                -1,
            ],
        )
        relative_position_bias = ops.transpose(relative_position_bias, axes=[2, 0, 1])
        attention_score += relative_position_bias[None,]
        attention_weight = self.softmax(attention_score)
        attention_weight = self.attention_dropout(attention_weight) #batch_, n_heads, token_length, token_length
        #value tensor shape : batch_, n_heads, token_length, dimension_per_heads(=C)
        attended_output = attention_weight@v
        attended_output = ops.transpose(attended_output, [0, 2, 1, 3])
        attended_output = ops.reshape(attended_output, [batch_, token_length, embed_dims])
        attended_output = self.projection_dropout(self.proj(attended_output))
        self.attention_weight = attention_weight
        if self.return_attention_weights:
            return attended_output, attention_weight
        else:
            return attended_output
        
        
class Block(keras.layers.Layer):
    def __init__(self, #이하는 Window Attention configurations
                 window_size, num_heads, global_query, 
                 qkv_bias = True, qk_scale = None, dropout_rate = 0.05, 
                 # 이하는 MLP module의 configuration
                 mlp_ratio = 4.0, layer_scale = None, return_attention_weights = False,
                 **kwargs):
        super().__init__(**kwargs)
        self.window_size = window_size
        self.n_heads = num_heads
        self.global_query = global_query
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout_rate = dropout_rate
        
        self.mlp_ratio = mlp_ratio
        self.layer_scale = layer_scale
        self.return_attention_weights = return_attention_weights
    def build(self, input_shape):
        #input tensor : list of key/query or key only
        # each tensor is batch_size, w, h, channel dims shape tensor
        batch_size, H, W, dims = input_shape[0]
        self.norm1 = keras.layers.LayerNormalization(epsilon = 1e-5)
        self.norm2 = keras.layers.LayerNormalization(epsilon = 1e-5)
        self.window_attention = WindowAttention(window_size = self.window_size,
                                               n_heads = self.n_heads,
                                               global_query = self.global_query,
                                               qkv_bias = self.qkv_bias,
                                               qk_scale = self.qk_scale,
                                               dropout_rate = self.dropout_rate,
                                                return_attention_weights = self.return_attention_weights)
        self.mlps = MLP(middle_dim = int(self.mlp_ratio * dims), dropout = self.dropout_rate)
        if self.layer_scale != None:
            self.gamma1 = self.add_weight(shape = [dims], name = "Gamma1", trainable = True,
                                         initializer = keras.initializer.Constant(self.layer_scale), dtype = self.dtype)
            self.gamma2 = self.add_weight(shape = [dims], name = "Gamma2", trainable = True,
                                         initializer = keras.initializer.Constant(self.layer_scale), dtype = self.dtype)
        else:
            self.gamma1, self.gamma2 = 1.0, 1.0
        self.n_windows = int(H//self.window_size) * int(W//self.window_size)
        
    #input feature map을 일정 크기의 window로 partition을 만들어주는 함수 및
    # 그 partition을 받아 원래의 feature map으로 돌려주는 함수를 만들자
    def window_partition(self, inputs): #feature map -> multiple windows
        batch_size, H, W, dims = ops.shape(inputs)
        h, w = H//self.window_size, W//self.window_size
        inputs = ops.reshape(inputs, [batch_size, 
                                      h, self.window_size,
                                     w, self.window_size, 
                                     dims])
        inputs = ops.transpose(inputs, [0,#batch_size
                                        1,3, #h, w
                                        2,4, #winsize, winsize
                                        5])
        return ops.reshape(inputs, [-1, self.window_size, self.window_size, dims]) #batch_size*n_windows, window_size, window_size, dims
        
    def window_reverse(self, inputs, H, W, dims): #window partition -> original feature map
        x = ops.reshape(inputs, [-1, H//self.window_size, W//self.window_size, self.window_size, self.window_size, dims])
        x = ops.transpose(x, [0, 1, 3, 2, 4, 5])
        return ops.reshape(x, [-1, H, W, dims])
    
    def call(self, inputs, **kwargs):
        if self.global_query:
            inputs, global_query = inputs
        else:
            inputs = inputs[0]
        batch_size, H, W, dims = ops.shape(inputs)
        x = self.norm1(inputs)
        x = self.window_partition(x) 
        x = ops.reshape(x, [-1, self.window_size*self.window_size, dims])
        if self.global_query:
            outputs_ = self.window_attention([x, global_query]
                                     )
        else:
            outputs_ = self.window_attention([x])
        if self.return_attention_weights:
            x, attention_weight = outputs_
        else:
            x = outputs_
        x = self.window_reverse(x, H, W, dims)
        x = inputs + self.gamma1*x
        x += self.gamma2*(self.mlps(self.norm2(x)))
        if self.return_attention_weights:
            return x, attention_weight
        else:
            return x
    
class Level(keras.layers.Layer):
    def __init__(self, 
                depth, #<- Block repetition depth
                num_heads, window_size, keepdims, #downsampler 및 block의 hyperparameter
                downsample = True, mlp_ratio = 4.0,
                qkv_bias = True, qk_scale = None,
                dropout = 0.05, layer_scale = None, return_attention_weights = True,
                **kwargs):
        super().__init__(**kwargs)
        self.depth = depth
        self.n_heads = num_heads
        self.window_size = window_size
        self.keepdims = keepdims
        self.downsample = downsample
        self.mlp_ratio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout_rate = dropout
        self.layer_scale = layer_scale
        self.return_attention_weights = return_attention_weights
        
    def build(self, input_shape):
        #input tensor : feature map / patches
        batch_size, H, W, dims = input_shape
        self.blocks = [Block(window_size = self.window_size, num_heads = self.n_heads, global_query = bool(idx%2), 
                             qkv_bias = self.qkv_bias, qk_scale = self.qk_scale, dropout_rate = self.dropout_rate, 
                             mlp_ratio = self.mlp_ratio, layer_scale = self.layer_scale, return_attention_weights = self.return_attention_weights,
                             name = f"GCViTBlock{idx+1}") for idx in range(self.depth)]
        self.downsampler = DownSampler(name = "Downsampler")
        self.query_generator = GlobalQueryGenerator(keepdims = self.keepdims, name = "GlobalQueryGenerator")
        
    def call(self, inputs, **kwargs):
        patches = inputs
        global_query = self.query_generator(inputs)
        for idx, block in enumerate(self.blocks):
            if idx % 2 :
                outputs_ = block([patches, global_query])
            else:
                outputs_ = block([patches])
            if self.return_attention_weights:
                patches, attention_weights = outputs_
            else:
                patches = outputs_
        if self.downsample == False:
            return patches
        else:
            return self.downsampler(patches)
def get_gcvit_configs(res, initial_embedding_dims, name = None):
    return {'res' : res,
            'embed_dims' : initial_embedding_dims,
            "patch_embedding_type" : "conv", #conv or tokenlearner
            "level_depth" : [2,4,6,8],
            "level_heads" : [2,4,8,16],
            "level_keepdims" : [[0,0,0],
                                   [0,0],
                                   [1], 
                                    [1]
                                   ], #3번째 level부터는 window attention == global attention
            "level_window_size" : [res//32, res//32, res//16, res//32],
            "model_name" : f"GCViT_res{res}" if name == None else name
                }
def get_gcvit(configs):
    res = configs["res"]
    inputs = Input([res,res,3], name = "ImageInput")
    patcher = PatchEmbedding(embed_dim = configs['embed_dims'], patching_type = configs["patch_embedding_type"],
                             name = "PatchEmbedding")
    patches = patcher(inputs)
    
    for idx, (depth, heads, keepdims, window_size) in enumerate(zip(configs["level_depth"], configs["level_heads"], configs["level_keepdims"], configs["level_window_size"])):
        if idx == len(configs['level_depth'])-1:
            downsample = False
        else:
            downsample = True
        level = Level(depth = depth, num_heads = heads, window_size = window_size, keepdims = keepdims, downsample = downsample,
                      name = f"GCViT_Lv{idx+1}_downsample_{downsample}")
        patches = level(patches)
    model = keras.Model(inputs, patches,
                       name = configs["model_name"])
    return model

--------------

# MLP-mixer
- for lower resource demand!
- Code Reference : [Keras.io](https://keras.io/examples/vision/mlp_image_classification/#the-mlpmixer-model)

![](https://velog.velcdn.com/images/minkyu4506/post/0237ee55-74eb-4836-952c-6bd33694aecc/image.png)

In [10]:
class NaivePatchesExtraction(keras.layers.Layer): #untrainable layer
    def __init__(self, patch_size, output_type = "seq", **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size
        self.output_type = output_type
        assert output_type in ["seq", "sequence", "Sequence", "3D", "conv", "feature_map", "2D"]
    def call(self, x):
        if len(ops.shape(x)) == 3:
            x = ops.expand_dims(x, 0)
        patches = keras.ops.image.extract_patches(x, self.patch_size)
        batch_size = keras.ops.shape(patches)[0]
        num_patches = keras.ops.shape(patches)[1] * keras.ops.shape(patches)[2]
        patch_dim = keras.ops.shape(patches)[3]
        if self.output_type in ["seq", "sequence", "Sequence", "3D"] : 
            out = keras.ops.reshape(patches, (batch_size, num_patches, patch_dim))
        elif self.output_type in ["conv", "feature_map", "2D"]:
            out = keras.ops.reshape(patches, (batch_size, keras.ops.shape(patches)[1], keras.ops.shape(patches)[2],
                                             patch_dim))
        return out

In [11]:
class MLPMixer(keras.layers.Layer): #untrainable layer
    def __init__(self, units = None, dropout_rate = 0.2, **kwargs):
        super().__init__(**kwargs)
        self.layernorm1 = keras.layers.LayerNormalization()
        self.layernorm2 = keras.layers.LayerNormalization()
        self.drop_rate = dropout_rate
        self.units = units
        
        
    def build(self, input_shape):
        #batch, seq_len, dims
        batch_size = input_shape[0]
        n_tokens = input_shape[1]
        channels = input_shape[2]
        embed_dim_middle = channels if self.units is None else self.units
        
        self.mlp1 = keras.Sequential([keras.layers.Dense(units = n_tokens, use_bias = False),
                                     keras.layers.Activation("gelu"),
                                     keras.layers.Dropout(self.drop_rate),
                                     keras.layers.Dense(units = n_tokens, use_bias = False),],
                                    name = "MLP1_TokenWiseMixing") #output : batch, embedding_dims(channels), n_tokens
        
        self.mlp2 = keras.Sequential([keras.layers.Dense(units = embed_dim_middle, use_bias = False),
                                     keras.layers.Activation("gelu"),
                                     keras.layers.Dropout(self.drop_rate),
                                     keras.layers.Dense(units = channels, use_bias = False),],
                                    name = "MLP2_ChannelWiseMixing")
    def enable_lora(self, rank):
        print("Use only in pretrained model!\n\n")
        if rank == 0:
            print("lora disabled\n\n")
            pass
        else:
            self.layernorm1.trainable = False
            self.layernorm2.trainable = False
            for layer in self.mlp1.layers:
                if isinstance(layer, keras.layers.Dense):
                    layer.enable_lora(rank)
                else:
                    layer.trainable = False
            for layer in self.mlp2.layers:
                if isinstance(layer, keras.layers.Dense):
                    layer.enable_lora(rank)
                else:
                    layer.trainable = False
    def call(self, x):
        # x : patches, [batch, n_patches, embed_dims]
        normed_patch = self.layernorm1(x)
        normed_patch = ops.transpose(normed_patch, [0,2,1]) #batch, embed_dims, n_patches
        normed_patch = self.mlp1(normed_patch)
        normed_patch = ops.transpose(normed_patch, [0,2,1]) #batch, n_patches, embed_dims
        normed_patch += x
        normed_patch2 = self.layernorm2(normed_patch)
        normed_patch2 = self.mlp2(normed_patch2)
        normed_patch2 += normed_patch
        return normed_patch2

In [12]:
def get_mlp_mixer(res, name = "mlpmixer_16_4_768"):
    _, patch_size, depth, embed_dims = name.split("_")
    patch_size = int(patch_size)
    depth = int(depth)
    embed_dims = int(embed_dims)
    
    model_name = f"Res{res}_{name}"
    inputs = Input([res,res,3], name = "ImageInput")
    scaled_input = inputs/255.0
    patches = NaivePatchesExtraction(patch_size, name = "PatchExtraction")(scaled_input)
    patches_embedding = keras_nlp.layers.RotaryEmbedding(name = "RotaryPositionalEmbedding")(patches)
    patches_embedding = keras.layers.Dense(units = embed_dims, name = "SimpleMLP_Channelwise")(patches_embedding)
    for idx in range(depth):
        patches_embedding = MLPMixer(name = f"MLPMixer_{idx+1}")(patches_embedding)
    model = Model(inputs, patches_embedding,
                 name = model_name)
    return model

# ConvMixer
- Dense layer inside MLPmixer --> Conv2D
- Code Reference : [Keras.io](https://keras.io/examples/vision/convmixer/)

![](https://i.imgur.com/yF8actg.png)

In [13]:
class ConvMixer(keras.layers.Layer): #untrainable layer
    def __init__(self, filters = None, kernel_size = 5, dropout_rate = 0.2, output_type = "2D", **kwargs):
        super().__init__(**kwargs)
        self.layernorm1 = keras.layers.LayerNormalization()
        self.layernorm2 = keras.layers.LayerNormalization()
        self.drop_rate = dropout_rate ; self.drop = keras.layers.Dropout(self.drop_rate)
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.output_type = output_type
        assert output_type in ["seq", "sequence", "Sequence", "3D", "conv", "feature_map", "2D"]
        
    def build(self, input_shape):
        #batch, seq_len, dims
        batch_size = input_shape[0]
        h = input_shape[1]
        w = input_shape[2]
        channels = input_shape[3]
        c = channels if self.filters is None else self.filters
        
        self.depthconv = keras.layers.DepthwiseConv2D(kernel_size = self.kernel_size, padding = "SAME", use_bias = False)
        self.pointconv = keras.layers.Conv2D(filters = c, kernel_size = 1, padding = 'SAME', use_bias = False)
        self.gelu = keras.layers.Activation('gelu')

    def call(self, x):
        # x : feature_map, [batch, h,w, embed_dims]
        fmap = self.depthconv(x)
        fmap = self.gelu(fmap)
        fmap = self.layernorm1(fmap) + x
        fmap = self.pointconv(fmap)
        fmap = self.gelu(fmap)
        fmap = self.layernorm2(fmap)
        
        batch_size = ops.shape(fmap)[0]
        h = ops.shape(fmap)[1]
        w = ops.shape(fmap)[2]
        dims_ = ops.shape(fmap)[3]
        
        if self.output_type in ["seq", "sequence", "Sequence", "3D"] : 
            fmap = keras.ops.reshape(fmap, (batch_size, h*w, dims_))
            return fmap
        elif self.output_type in ["conv", "feature_map", "2D"]:
            return fmap

In [14]:
def get_conv_mixer(res, name = "convmixer_16_4_768"):
    _, patch_size, depth, embed_dims = name.split("_")
    patch_size = int(patch_size)
    depth = int(depth)
    embed_dims = int(embed_dims)
    
    model_name = f"Res{res}_{name}"
    inputs = Input([res,res,3], name = "ImageInput")
    scaled_input = inputs/255.0
    patches = NaivePatchesExtraction(patch_size, name = "PatchExtraction")(scaled_input)
    patches_embedding = keras_nlp.layers.RotaryEmbedding(name = "RotaryPositionalEmbedding")(patches)
    batch, n_patches, dims = ops.shape(patches_embedding) ; res_ = int(ops.sqrt(ops.cast(n_patches, "float32")))
    patches_embedding = ops.reshape(patches_embedding, (-1, res_, res_, dims))
    
    patches_embedding = keras.layers.Dense(units = embed_dims, name = "SimpleMLP_Channelwise")(patches_embedding)
    for idx in range(depth):
        if idx == depth-1:
            types = "seq"
        else:
            types = "2D"
        patches_embedding = ConvMixer(name = f"ConvMixer_{idx+1}", output_type = types)(patches_embedding)
    model = Model(inputs, patches_embedding,
                 name = model_name)
    return model

-----------

# Get final Feature extractor
- according to [recent study](https://arxiv.org/abs/2309.16588), add cls token and register tokens "after" the patches!

In [15]:
def get_feature_extractor(conv_base, #if None, Vanilla ViT
                         embed_dims, res, pe_type = "rotary",
                          patch_size = 16,
                        att_depth = 4, att_heads = 16) : 
    inputs = Input([res,res,3], name = "Input_images")
    batch_size = ops.shape(inputs)[0]
    if conv_base in [None, 'vit', "ViT"] : #Vanilla Vision Transformer
        scaled_inputs = keras.layers.LayerNormalization(name = "InitialLN")(inputs)
        patches = Conv2D(filters = embed_dims, activation = "gelu", kernel_size = patch_size, strides = patch_size, padding = 'SAME', name = "PatchingStem")(scaled_inputs)
        _, w, h, dims = ops.shape(patches) ; n_patches = w*h 
        
        patches = ops.reshape(patches, [-1, w*h, dims])
        if pe_type in ['rotary', 'rotation', 'rotatory', 'roformer']:
            patches = keras_nlp.layers.RotaryEmbedding(name = "RotaryPositionalEmbedding")(patches)
        elif pe_type in ["learnable", 'absolute']:
            patches = PatchEncoder(num_patches = w*h, projection_dim = embed_dims)(patches)
        elif pe_type == None:
            pass
        
        cls_token = ops.expand_dims(keras.layers.GlobalAveragePooling1D(name = 'GAPforClsToken')(patches),
                                    axis = 1)
        register_tokens = ops.tile(ops.ones_like(cls_token), 
                                   [1,2,1])
        patches = ops.concatenate([patches, cls_token, register_tokens], axis = 1)
    
        for idx in range(att_depth):
            x0 = LayerNormalization(name = f"PreLN{idx+1}")(patches)
            x1, attention_score = AttentionPooling(att_heads, embed_dims, name = f"MHA_after_Conv_{idx+1}")([x0, x0])
            x2 = keras.layers.Add(name = f"PreAdd{idx+1}")([patches, x1])
            x3 = LayerNormalization(name = f"PostLN{idx+1}")(x2)
            x4 = Dense(units = embed_dims, activation = 'gelu', name = f"TokenMixMLP{idx+1}")(x3)
            patches = keras.layers.Add(name = "Encoded_Patches" if idx == att_depth-1 else f"PostAdd{idx+1}")([x4, x2])
        learned_token = keras.layers.Identity(name = "feature_vector")(patches[:, -3, :])
        patches = keras.layers.Identity(name = "encoded_patches")(patches[:, :n_patches, :])
        attention_score = attention_score[:, :, n_patches, :-3]
        attention_score = ops.expand_dims(attention_score, axis = -2)
        attention_score = keras.layers.Identity(name = "attention_weight")(attention_score) 
        model_name = f"ViT_depth{att_depth}_dims{embed_dims}_heads{att_heads}_patch{patch_size}"
    else:
        feature_map = conv_base(inputs)
        if len(ops.shape(feature_map)) == 4:
            _, w, h, dims = ops.shape(feature_map)
            dims = ops.shape(feature_map)[-1] ; batch_size = ops.shape(feature_map)[0]
            feature_map = ops.reshape(feature_map, [-1, w*h, dims])
        n_patches = ops.shape(feature_map)[1]
        if pe_type in ['rotary', 'rotation', 'rotatory', 'roformer']:
            feature_map = keras_nlp.layers.RotaryEmbedding(name = "RotaryPositionalEmbedding")(feature_map)
        elif pe_type in ["learnable", 'absolute']:
            feature_map = PatchEncoder(num_patches = n_patches, projection_dim = embed_dims)(feature_map)
        elif pe_type == None:
            pass

        learned_token = keras.layers.GlobalAveragePooling1D(name = 'GAPforRepVec')(feature_map)
        learned_token = learned_token[:, tf.newaxis, :]
        register_tokens = ops.tile(ops.ones_like(learned_token), 
                                   [1,2,1])
        feature_map = ops.concatenate([feature_map,
                                      learned_token,
                                      register_tokens], axis = 1)
        
        for idx in range(att_depth):
            feature_map, attention_score = AttentionPooling(att_heads, embed_dims, name = f"MHA_after_Conv_{idx+1}")([feature_map, feature_map])
        learned_token = keras.layers.Identity(name = "feature_vector")(feature_map[:, -3, :])
        attention_score = attention_score[:, :, n_patches, :-3]
        attention_score = ops.expand_dims(attention_score, axis = -2)
        attention_score = keras.layers.Identity(name = "attention_weight")(attention_score) 
        patches = keras.layers.Identity(name = "encoded_patches")(feature_map[:, :n_patches, :])
        
        model_name = f"{conv_base.name}_depth{att_depth}_dims{embed_dims}_heads{att_heads}"
    
    model = Model(inputs, [learned_token, attention_score],
                  name = model_name)
    return model

def get_full_model(conv_base_name, res, embed_dims = 1280, patch_size = 16, pe_type = 'rotary',
                   att_depth = 4, att_heads = 8,
                  extra_configs = None):
    if conv_base_name in ["effnet", 'EfficientNet']:
        conv_base = keras.applications.EfficientNetV2B1(input_shape = [res,res,3],
                                                       include_top = False)
    elif conv_base_name in ["effnet_small", "EfficientNetSmall"]:
        conv_base = keras.applications.EfficientNetV2S(input_shape = [res,res,3],
                                                       include_top = False)
    elif conv_base_name in ["effnet_base", "EfficientNetBase"]:
        conv_base = keras.applications.EfficientNetV2M(input_shape = [res,res,3],
                                                       include_top = False)
    elif conv_base_name in ["convnext", 'ConvNeXt']:
        conv_base = keras.applications.ConvNeXtTiny(input_shape = [res,res,3],
                                                       include_top = False)
    elif conv_base_name in ["convnext_small", 'ConvNeXtSmall']:
        conv_base = keras.applications.ConvNeXtSmall(input_shape = [res,res,3],
                                                       include_top = False)
    elif conv_base_name in ["convnext_base", 'ConvNeXtBase']:
        conv_base = keras.applications.ConvNeXtBase(input_shape = [res,res,3],
                                                       include_top = False)
    elif isinstance(conv_base_name, dict):
        conv_base = get_gcvit(conv_base_name)
    elif conv_base_name.split("_")[0] in ["MLPMixer", "mlpmixer", "MLP_mixer", "mlp_mixer"]:
        conv_base = get_mlp_mixer(res = res, name = conv_base_name)
        pe_type = None
    elif conv_base_name.split("_")[0] in ["ConvMixer", "convmixer", "Conv_mixer", "conv_mixer"]:
        conv_base = get_conv_mixer(res = res, name = conv_base_name)
        pe_type = None
    else:
        conv_base = None
    return get_feature_extractor(conv_base, pe_type = pe_type, res = res, patch_size = patch_size, embed_dims = embed_dims, att_depth = att_depth, att_heads = att_heads)

---------
# Information-maximization method

In [16]:
#helper functions
# For Barlow Twins
def get_cor_matrix(z1, z2, e = 1e-5):
    #z1, z2 = [batch, embed_dims] shape tensor
    # 각각 Normalize한 뒤 내적 -> dims by dims correlation matrix
    z1_mean, z1_std = ops.mean(z1, axis = 0), ops.std(z1, axis = 0)
    z2_mean, z2_std = ops.mean(z2, axis = 0), ops.std(z2, axis = 0)
    z1 = (z1 - z1_mean) / (z1_std + e)
    z2 = (z2 - z2_mean) / (z2_std + e)
    bs = tf.cast(ops.shape(z1)[0], tf.float32)
    matrix = (ops.transpose(z1)@z2) / bs
    matrix = tf.cast(matrix, tf.float32)
    return matrix

# For VICreg
def invariance_loss(za, zb): #invariance
    l2_distances = keras.losses.MeanSquaredError(reduction = None)(za, zb)
    return ops.cast(l2_distances, "float32")

def variance(za, e = 1e-4, gamma = 5.0):
    mu, var = tf.nn.moments(za, axes = 0)
    mu = tf.cast(mu, tf.float32)
    var = tf.cast(var, tf.float32)
    var += tf.cast(e, tf.float32)
    
    s_xe_hinge = gamma - tf.math.sqrt(var)
    s_val = tf.math.maximum(0.0, s_xe_hinge)
    return ops.cast(s_val, "float32")

def covariance(za, e = 1e-4, testing = False):
    mu, var = tf.nn.moments(za, axes = 0) ; bs = tf.cast(ops.shape(za)[0], tf.float32) ; embed_dims = tf.cast(ops.shape(za)[-1], tf.float32)
    mu = tf.cast(mu, tf.float32)
    var = tf.cast(var, tf.float32)
    div_ = ops.maximum(bs-1, 1)
    
    za -= mu #batch, dims
    cov_mat = tf.transpose(za)@za/div_
    lower_tri = cov_mat - tf.linalg.band_part(cov_mat, 0, -1)
    upper_tri = cov_mat - tf.linalg.band_part(cov_mat, -1, 0)
    off_diag = lower_tri + upper_tri
    off_diag = ops.sum(tf.math.square(off_diag))/embed_dims
    off_diag = ops.cast(off_diag, "float32")
    if testing : 
        print(f"Covariacne matrix shape : {ops.shape(cov_mat)}")
    return off_diag

In [17]:
class BarlowModel(keras.Model):
    def __init__(self, feature_extractor, embed_dims, probe = False, multiview = False,
                 probe_heads = None, probe_activation = "sigmoid",
                 diag = 0.6, off_diag = 0.4, 
                 **kwargs):
        super().__init__(**kwargs)
        self.embed_dims = embed_dims
        self.multiview = multiview
        self.feature_extractor = feature_extractor #weight sharing for VICreg, Barlow Twins
        self.projector_a = self.get_projector()
        self.projector_b = self.get_projector()
        self.diag = diag ; self.off_diag = off_diag
        self.loss_tracker = tf.keras.metrics.Mean("Barlow_loss_tracker")
        self.linear_probing = probe
        if probe:
            self.probe_loss_tracker = tf.keras.metrics.Mean("Probe_loss_tracker")
            self.probe_metrics = [
                                  keras.metrics.F1Score(average="micro", threshold=None, name="probe_micro_f1_score"),
                                 keras.metrics.TruePositives(name = "probe_TP"),
                                 keras.metrics.TrueNegatives(name = "probe_TN"),
                                 keras.metrics.FalsePositives(name = "probe_FP"),
                                 keras.metrics.FalseNegatives(name = "probe_FN")]
            
            self.probe_categories = probe_heads
            self.probe_act = probe_activation
            self.linear_probe = keras.layers.Dense(units = probe_heads, activation = self.probe_act, dtype = "float32")
            if self.probe_act == "sigmoid":
                self.probe_loss_fn = keras.losses.BinaryCrossentropy(reduction = None, label_smoothing = 0.1)
                self.probe_metrics.append(keras.metrics.BinaryAccuracy(name="probe_accuracy", threshold=0.5))
            elif self.probe_act == "softmax":
                self.probe_loss_fn = keras.losses.CategoricalCrossentropy(reduction = None, label_smoothing = 0.1)
                self.probe_metrics.append(keras.metrics.CategoricalAccuracy(name="probe_accuracy", threshold=0.5))
    def compile(self, optimizer, probe_optimizer = None, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer
        if self.linear_probing:
            self.probe_optimizer = probe_optimizer
        
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
                "Multiview(>2)" : self.multiview,
               'diag_part_coefficient' : self.diag,
               'off_diag_coefficient' : self.off_diag,
               "SSL_method" : "Barlow_Twins",
               "Linear Probe" : self.linear_probing,
               "N_Categories" : self.probe_categories if self.linear_probing else 0,
               "Probe Activation" : self.probe_act if self.linear_probing else "NA"}
    
    def get_projector(self):
        
        model = keras.Sequential([Dense(units = self.embed_dims),
                                 Dense(units = self.embed_dims),
                                 Dense(units = self.embed_dims, dtype = "float32")]
                                )
        return model
    
    def compute_loss(self, correlation_matrix):
        diag_component = tf.linalg.diag_part(correlation_matrix)
        zero_diag = tf.zeros(correlation_matrix.shape[-1])
        off_diag_matrix = tf.linalg.set_diag(correlation_matrix, zero_diag)
        
        diag_loss = tf.pow(diag_component-1, 2) * self.diag
        off_diag_loss = tf.pow(off_diag_matrix, 2) * self.off_diag
        loss = tf.reduce_mean(diag_loss) + tf.reduce_mean(off_diag_loss)
        loss = tf.cast(loss, tf.float32)
        
        return loss
        
    def train_step(self, dataset):
        
        if self.linear_probing:
            data, aug_data, labels = dataset
        else:
            if self.multiview:
                data = dataset[0]
                aug_data = dataset[1:] ; n_augs = len(aug_data)
                aug_data = ops.concatenate(aug_data, axis = 0)
            else:
                data, aug_data = dataset
        ## 1. SSL encoder loss ##
        with tf.GradientTape() as tape:
            if len(self.feature_extractor.outputs) == 2 :
                feature_seq, weights = self.feature_extractor(data)
                feature_seq_aug, weights_aug = self.feature_extractor(aug_data)
            elif len(self.feature_extractor.outputs) == 1:
                feature_seq = self.feature_extractor(data)
                feature_seq_aug = self.feature_extractor(aug_data)
                
            if len(ops.shape(feature_seq)) == 3: 
                rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
                rep_vector_aug = keras.layers.GlobalAveragePooling1D()(feature_seq_aug)
            elif len(ops.shape(feature_seq)) == 4:
                rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
                rep_vector_aug = keras.layers.GlobalAveragePooling2D()(feature_seq_aug)
            elif len(ops.shape(feature_seq)) == 2:
                rep_vector = feature_seq
                rep_vector_aug = feature_seq_aug
            original_rep_vector = rep_vector #for Linear probing
            
            rep_vector = self.projector_a(rep_vector)
            rep_vector_aug = self.projector_b(rep_vector_aug)
            if self.multiview:
                rep_vector_aug_set = ops.split(rep_vector_aug, n_augs, 0)
                loss = []
                for idx in range(n_augs):
                    correlation_matrix = get_cor_matrix(rep_vector, rep_vector_aug_set[idx])
                    loss.append(self.compute_loss(correlation_matrix)
                               )
                loss = ops.mean(loss)
            else:
                correlation_matrix = get_cor_matrix(rep_vector, rep_vector_aug)
                loss = self.compute_loss(correlation_matrix)
    

        gradients = tape.gradient(loss, 
                                 (self.feature_extractor.trainable_variables + self.projector_a.trainable_variables + self.projector_b.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients, 
                                          (self.feature_extractor.trainable_variables + self.projector_a.trainable_variables + self.projector_b.trainable_variables)
                                          ))
        self.loss_tracker.update_state(loss)
        output_dict = {'ssl_loss' : self.loss_tracker.result()}
        if self.linear_probing:
            ##2. linear probing and calculate metrics##
            
            with tf.GradientTape() as tape:
                if len(self.feature_extractor.outputs) == 2 :
                    feature_seq, weights = self.feature_extractor(data, training = False)
                elif len(self.feature_extractor.outputs) == 1:
                    feature_seq = self.feature_extractor(data, training = False)
                if len(ops.shape(feature_seq)) == 3: 
                    original_rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
                elif len(ops.shape(feature_seq)) == 4:
                    original_rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
                elif len(ops.shape(feature_seq)) == 2:
                    rep_vector = feature_seq
        
                class_logits = self.linear_probe(original_rep_vector, training = True)
                class_probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
                class_probe_loss = ops.mean(class_probe_loss)
            gradients = tape.gradient(class_probe_loss, 
                                            self.linear_probe.trainable_weights)
            self.probe_optimizer.apply_gradients(zip(gradients, self.linear_probe.trainable_weights))
            self.probe_loss_tracker.update_state(class_probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            #metrics
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
        else:
            pass
        del tape
        return output_dict
    def test_step(self, dataset):
        if self.linear_probing:
            data, aug_data, labels = dataset
        else:
            if self.multiview:
                data = dataset[0]
                aug_data = dataset[1:] ; n_augs = len(aug_data)
                aug_data = ops.concatenate(aug_data, axis = 0)
            else:
                data, aug_data = dataset
            
        if len(self.feature_extractor.outputs) == 2 :
            feature_seq, weights = self.feature_extractor(data)
            feature_seq_aug, weights_aug = self.feature_extractor(aug_data)
        elif len(self.feature_extractor.outputs) == 1:
            feature_seq = self.feature_extractor(data)
            feature_seq_aug = self.feature_extractor(aug_data)
        if len(ops.shape(feature_seq)) == 3: 
            rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
            rep_vector_aug = keras.layers.GlobalAveragePooling1D()(feature_seq_aug)
        elif len(ops.shape(feature_seq)) == 4:
            rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
            rep_vector_aug = keras.layers.GlobalAveragePooling2D()(feature_seq_aug)
        elif len(ops.shape(feature_seq)) == 2:
            rep_vector = feature_seq
            rep_vector_aug = feature_seq_aug
        
        rep_vector = self.projector_a(rep_vector)
        rep_vector_aug = self.projector_b(rep_vector_aug)
        
        if self.multiview:
            rep_vector_aug_set = ops.split(rep_vector_aug, n_augs, 0)
            loss = []
            for idx in range(n_augs):
                correlation_matrix = get_cor_matrix(rep_vector, rep_vector_aug_set[idx])
                loss.append(self.compute_loss(correlation_matrix)
                            )
            loss = ops.mean(loss)
        else:
            correlation_matrix = get_cor_matrix(rep_vector, rep_vector_aug)
            loss = self.compute_loss(correlation_matrix)
        
        self.loss_tracker.update_state(loss)
        output_dict = {'ssl_loss' : self.loss_tracker.result()}
        
        if self.linear_probing:
            ##2. linear probing and calculate metrics##
            if len(self.feature_extractor.outputs) == 2 :
                feature_seq, weights = self.feature_extractor(data, training = False)
            elif len(self.feature_extractor.outputs) == 1:
                feature_seq = self.feature_extractor(data, training = False)
            if len(ops.shape(feature_seq)) == 3: 
                original_rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
            elif len(ops.shape(feature_seq)) == 4:
                original_rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
            elif len(ops.shape(feature_seq)) == 2:
                original_rep_vector = feature_seq

            class_logits = self.linear_probe(original_rep_vector, training = False)
            class_probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
            class_probe_loss = ops.mean(class_probe_loss)
            self.probe_loss_tracker.update_state(class_probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            #metrics
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
        else:
            pass
        return output_dict
    def get_classifier_model(self):
        self.feature_extractor.trainable = True
        if self.linear_probing == False:
            return self.feature_extractor
        self.linear_probe.trainable = True
        inputs = self.feature_extractor.input
        outputs = self.feature_extractor.outputs
        if len(self.feature_extractor.outputs) == 2 :
            feature_map, att_weights = outputs
        elif len(self.feature_extractor.outputs) == 1:
            feature_map = outputs[0]
        
        if len(ops.shape(feature_map)) == 3: 
            rep_vector = keras.layers.GlobalAveragePooling1D()(feature_map)
        elif len(ops.shape(feature_map)) == 4:
            rep_vector = keras.layers.GlobalAveragePooling2D()(feature_map)
        elif len(ops.shape(feature_map)) == 2:
            rep_vector = feature_map
            
        class_logits = self.linear_probe(rep_vector)
        classifier_model = keras.Model(inputs, class_logits,
                                      name = f"SSL_pretrained_{self.feature_extractor.name}")
        return classifier_model
    def call(self, dataset):
        return self.test_step(dataset)

In [18]:
class VICRegModel(keras.Model):
    def __init__(self, feature_extractor, embed_dims, multiview = False,
                 probe = False, 
                 probe_heads = None, probe_activation = "sigmoid",
                 variance_coeff = 20, invariance_coeff = 20, covariance_coeff = 1, 
                 variance_gamma = 5.0,
                 **kwargs):
        super().__init__(**kwargs)
        self.embed_dims = embed_dims
        self.multiview = multiview
        self.feature_extractor = feature_extractor #weight sharing for VICreg, Barlow Twins
        self.projector_a = self.get_projector()
        self.projector_b = self.get_projector()
        self.var_coef = variance_coeff ; self.invar_coef = invariance_coeff ; self.cov_coef = covariance_coeff
        self.gamma = variance_gamma
        
        self.loss_tracker = tf.keras.metrics.Mean("VIC_loss_tracker")
        self.invar_loss_tracker = tf.keras.metrics.Mean("Invariance_loss_tracker")
        self.var_loss_tracker = tf.keras.metrics.Mean("Variance_loss_tracker")
        self.covar_loss_tracker = tf.keras.metrics.Mean("Covariance_loss_tracker")
        self.linear_probing = probe
        if probe:
            self.probe_loss_tracker = tf.keras.metrics.Mean("Probe_loss_tracker")
            self.probe_metrics = [keras.metrics.Accuracy(name="probe_accuracy"),
                                  keras.metrics.F1Score(average="micro", threshold=None, name="probe_micro_f1_score"),
                                 keras.metrics.TruePositives(name = "probe_TP"),
                                 keras.metrics.TrueNegatives(name = "probe_TN"),
                                 keras.metrics.FalsePositives(name = "probe_FP"),
                                 keras.metrics.FalseNegatives(name = "probe_FN")]
            
            self.probe_categories = probe_heads
            self.probe_act = probe_activation
            self.linear_probe = keras.layers.Dense(units = probe_heads, activation = self.probe_act)
            if self.probe_act == "sigmoid":
                self.probe_loss_fn = keras.losses.BinaryCrossentropy(reduction = None, label_smoothing = 0.1)
            elif self.probe_act == "softmax":
                self.probe_loss_fn = keras.losses.CategoricalCrossentropy(reduction = None, label_smoothing = 0.1)
    def compile(self, optimizer, probe_optimizer = None, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer
        if self.linear_probing:
            self.probe_optimizer = probe_optimizer     
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
                 "Multiview(>2)" : self.multiview,
               'Variance_coefficient' : self.var_coef,
               'Invariance_coefficient' : self.invar_coef,
               "Covariance_coefficient" : self.cov_coef,
                "Variance_gamma" : self.gamma,
               "SSL_method" : "VICReg",
               "Linear Probe" : self.linear_probing,
               "N_Categories" : self.probe_categories if self.linear_probing else 0,
               "Probe Activation" : self.probe_act if self.linear_probing else "NA"}
    def get_projector(self):
        model = keras.Sequential([Dense(units = self.embed_dims),
                                 Dense(units = self.embed_dims),
                                 Dense(units = self.embed_dims,dtype = "float32")]
                                )
        return model
    
    def compute_loss(self, rep_a, rep_b):
        invar_loss = invariance_loss(rep_a, rep_b) * self.invar_coef
        invar_loss = ops.mean(invar_loss)
        
        variance_a = variance(rep_a, gamma = self.gamma)
        variance_a = ops.mean(variance_a)
        
        covariance_a = covariance(rep_a)
        
        variance_b = variance(rep_b, gamma = self.gamma)
        variance_b = ops.mean(variance_b)
        
        covariance_b = covariance(rep_b)
        #Variance loss -> variance가 toward gamma
        # covariance -> covariance가 toward zero
        var_loss = (variance_a + variance_b) * self.var_coef
        covar_loss = (covariance_a + covariance_b) * self.cov_coef
        loss = invar_loss + var_loss + covar_loss
        return loss, invar_loss, var_loss, covar_loss
        
    def train_step(self, dataset):
        if self.linear_probing:
            data, aug_data, labels = dataset
        else:
            if self.multiview:
                data = dataset[0]
                aug_data = dataset[1:] ; n_augs = len(aug_data)
                aug_data = ops.concatenate(aug_data, axis = 0)
            else:
                data, aug_data = dataset
            
        with tf.GradientTape() as tape:
            
            if len(self.feature_extractor.outputs) == 2 :
                feature_seq, weights = self.feature_extractor(data)
                feature_seq_aug, weights_aug = self.feature_extractor(aug_data)
            elif len(self.feature_extractor.outputs) == 1:
                feature_seq = self.feature_extractor(data)
                feature_seq_aug = self.feature_extractor(aug_data)
            
            if len(ops.shape(feature_seq)) == 3: 
                rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
                rep_vector_aug = keras.layers.GlobalAveragePooling1D()(feature_seq_aug)
            elif len(ops.shape(feature_seq)) == 4:
                rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
                rep_vector_aug = keras.layers.GlobalAveragePooling2D()(feature_seq_aug)
            elif len(ops.shape(feature_seq)) == 2:
                rep_vector = feature_seq
                rep_vector_aug = feature_seq_aug
            
            
            rep_vector = self.projector_a(rep_vector)
            rep_vector_aug = self.projector_b(rep_vector_aug)
            if self.multiview:
                loss, invar_loss, var_loss, covar_loss = 0.0, 0.0, 0.0, 0.0
                rep_vector_aug_set = ops.split(rep_vector_aug, n_augs, 0)
                for idx in range(n_augs):
                    loss_, invar_loss_, var_loss_, covar_loss_ = self.compute_loss(rep_vector, rep_vector_aug_set[idx])
                    loss += loss_
                    invar_loss += invar_loss_
                    var_loss += var_loss_
                    covar_loss += covar_loss_
                loss /= n_augs
                invar_loss /= n_augs
                var_loss /= n_augs
                covar_loss /= n_augs
                
            else:
                loss, invar_loss, var_loss, covar_loss = self.compute_loss(rep_vector, rep_vector_aug)
        
        feature_extractor_var = self.feature_extractor.trainable_variables
        proj_a_var = self.projector_a.trainable_variables
        proj_b_var = self.projector_b.trainable_variables
        
        trainable_variables = (feature_extractor_var + proj_a_var + proj_b_var)
        gradients = tape.gradient(loss, trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, trainable_variables))
        
        self.loss_tracker.update_state(loss)
        self.invar_loss_tracker.update_state(invar_loss)
        self.var_loss_tracker.update_state(var_loss)
        self.covar_loss_tracker.update_state(covar_loss)
        output_dict = {'loss' : self.loss_tracker.result(),
                        "invariance_loss" : self.invar_loss_tracker.result(),
                        'variance_loss' : self.var_loss_tracker.result(),
                        'covariance_loss' : self.covar_loss_tracker.result()
                       }
        
        if self.linear_probing:
            ##2. linear probing and calculate metrics##
            
            with tf.GradientTape() as tape:
                if len(self.feature_extractor.outputs) == 2 :
                    feature_seq, weights = self.feature_extractor(data, training = False)
                elif len(self.feature_extractor.outputs) == 1:
                    feature_seq = self.feature_extractor(data, training = False)
                if len(ops.shape(feature_seq)) == 3: 
                    original_rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
                elif len(ops.shape(feature_seq)) == 4:
                    original_rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
                elif len(ops.shape(feature_seq)) == 2:
                    original_rep_vector = feature_seq
                    
                class_logits = self.linear_probe(original_rep_vector, training = True)
                class_probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
                class_probe_loss = ops.mean(class_probe_loss)
            gradients = tape.gradient(class_probe_loss, 
                                            self.linear_probe.trainable_weights)
            self.probe_optimizer.apply_gradients(zip(gradients, self.linear_probe.trainable_weights))
            self.probe_loss_tracker.update_state(class_probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            #metrics
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
        else:
            pass
        del tape
        return output_dict
    def test_step(self, dataset):
        if self.linear_probing:
            data, aug_data, labels = dataset
        else:
            if self.multiview:
                data = dataset[0]
                aug_data = dataset[1:] ; n_augs = len(aug_data)
                aug_data = ops.concatenate(aug_data, axis = 0)
            else:
                data, aug_data = dataset
        
        if len(self.feature_extractor.outputs) == 2 :
            feature_seq, weights = self.feature_extractor(data)
            feature_seq_aug, weights_aug = self.feature_extractor(aug_data)
        elif len(self.feature_extractor.outputs) == 1:
            feature_seq = self.feature_extractor(data)
            feature_seq_aug = self.feature_extractor(aug_data)
        
        if len(ops.shape(feature_seq)) == 3: 
            rep_vector = keras.layers.GlobalAveragePooling1D()(feature_seq)
            rep_vector_aug = keras.layers.GlobalAveragePooling1D()(feature_seq_aug)
        elif len(ops.shape(feature_seq)) == 4:
            rep_vector = keras.layers.GlobalAveragePooling2D()(feature_seq)
            rep_vector_aug = keras.layers.GlobalAveragePooling2D()(feature_seq_aug)
        elif len(ops.shape(feature_seq)) == 2:
            rep_vector = feature_seq
            rep_vector_aug = feature_seq_aug
            
        original_rep_vector = rep_vector
        
        rep_vector = self.projector_a(rep_vector, training = False)
        rep_vector_aug = self.projector_b(rep_vector_aug, training = False)
        if self.multiview:
            loss, invar_loss, var_loss, covar_loss = [],[],[],[]
            rep_vector_aug_set = ops.split(rep_vector_aug, n_augs, 0)
            for idx in range(n_augs):
                loss_, invar_loss_, var_loss_, covar_loss_ = self.compute_loss(rep_vector, rep_vector_aug_set[idx])
                loss.append(loss_)
                invar_loss.append(invar_loss_)
                var_loss.append(var_loss_)
                covar_loss.append(covar_loss_)
            loss = ops.mean(loss)
            invar_loss = ops.mean(invar_loss)
            var_loss = ops.mean(var_loss)
            covar_loss = ops.mean(covar_loss)    
        else:
            loss, invar_loss, var_loss, covar_loss = self.compute_loss(rep_vector, rep_vector_aug)
        
        self.loss_tracker.update_state(loss)
        self.invar_loss_tracker.update_state(invar_loss)
        self.var_loss_tracker.update_state(var_loss)
        self.covar_loss_tracker.update_state(covar_loss)
        
        output_dict =  {'loss' : self.loss_tracker.result(),
                "invariance_loss" : self.invar_loss_tracker.result(),
                'variance_loss' : self.var_loss_tracker.result(),
                'covariance_loss' : self.covar_loss_tracker.result()
               }
        if self.linear_probing:
            ##2. linear probing and calculate metrics##
            self.feature_extractor.trainable = False
            class_logits = self.linear_probe(original_rep_vector, training = False)
            class_probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
            class_probe_loss = ops.mean(class_probe_loss)
            self.probe_loss_tracker.update_state(class_probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            #metrics
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
            self.feature_extractor.trainable = True
        return output_dict
    def get_classifier_model(self):
        self.feature_extractor.trainable = True
        if self.linear_probing == False:
            return self.feature_extractor
        self.linear_probe.trainable = True
        inputs = self.feature_extractor.input
        outputs = self.feature_extractor.outputs
        if len(self.feature_extractor.outputs) == 2 :
            feature_map, att_weights = outputs
        elif len(self.feature_extractor.outputs) == 1:
            feature_map = outputs[0]
        
        if len(ops.shape(feature_map)) == 3: 
            rep_vector = keras.layers.GlobalAveragePooling1D()(feature_map)
        elif len(ops.shape(feature_map)) == 4:
            rep_vector = keras.layers.GlobalAveragePooling2D()(feature_map)
        elif len(ops.shape(feature_seq)) == 2:
            rep_vector = feature_map
            
        class_logits = self.linear_probe(rep_vector)
        classifier_model = keras.Model(inputs, class_logits,
                                      name = f"SSL_pretrained_{self.feature_extractor.name}")
        return classifier_model
    def call(self, dataset):
        return self.test_step(dataset)

-------------
# Contrastive-based method
- SimSiam
- SimCLR
- SwAV
- DINO
- MoCo and moco-based learnings:
    - [Dense Contrastive learning](https://arxiv.org/abs/2011.09157v2) -> DCL
    - [Nearest-Neighbor Contrastive Learning of Visual Representations](https://arxiv.org/abs/2104.14548v2) -> NNCLR

In [19]:
class SimSiam(keras.Model):
    def __init__(self, feature_extractor, embed_dims = 256, multiview = False,
                 probe = False, probe_heads = None, probe_activation = "sigmoid", **kwargs):
        super().__init__(**kwargs)
        self.feature_extractor = feature_extractor #attentive pooling을 거친 encoder로 가정 : rep_vector, attention_Weight를 output으로
        self.embed_dims = embed_dims
        self.multiview = multiview
        
        self.train_type = 'SimSiam'
        self.loss_tracker = keras.metrics.Mean(name = f"{self.train_type}_loss")
        self.probe = probe ; self.probe_heads = probe_heads
        self.probe_activation = probe_activation
        
        print("Train with Gradient Accumulation is recommended!")
        
        self.embed_dims = embed_dims
        self.predictor = keras.Sequential([keras.layers.Dense(units = self.embed_dims,
                                                            use_bias = False,
                                                            kernel_regularizer = keras.regularizers.l2(5e-4),
                                                            ),
                                          keras.layers.Activation("gelu"),
                                          keras.layers.BatchNormalization(),
                                          keras.layers.Dense(units = self.embed_dims),
                                          ], name = f"{self.train_type}_predictor")
        if self.probe:
            self.linear_probe = Dense(units = self.probe_heads, 
                                     activation = self.probe_activation)
            self.probe_loss_tracker = tf.keras.metrics.Mean("Probe_loss_tracker")
            self.probe_metrics = [keras.metrics.Accuracy(name="probe_accuracy"),
                                  keras.metrics.F1Score(average="micro", threshold=None, name="probe_micro_f1_score"),
                                 keras.metrics.TruePositives(name = "probe_TP"),
                                 keras.metrics.TrueNegatives(name = "probe_TN"),
                                 keras.metrics.FalsePositives(name = "probe_FP"),
                                 keras.metrics.FalseNegatives(name = "probe_FN")]
            if self.probe_activation == "sigmoid":
                self.probe_loss_fn = keras.losses.BinaryCrossentropy(reduction = None, label_smoothing = 0.1)
            elif self.probe_activation == "softmax":
                self.probe_loss_fn = keras.losses.CategoricalCrossentropy(reduction = None, label_smoothing = 0.1)
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
                "Multiview(>2)" : self.multiview,
               "SSL_method" : "SimSiam",
               "Linear Probe" : self.probe,
               "N_Categories" : self.probe_categories if self.probe else 0,
               "Probe Activation" : self.probe_act if self.probe else "NA"}
    def compile(self, optimizer, probe_optimizer = None, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer
        if self.probe:
            self.probe_optimizer = probe_optimizer 
    def compute_loss(self, p, z, epsilon = 1e-5):
        # stop gradient is essential in SimSiam structure
        # p, z is representation vectors : batch_size, embed_dims
        
        z = ops.stop_gradient(p)
        z = keras.utils.normalize(z, axis = -1, order = 2)
        p = keras.utils.normalize(p, axis = -1, order = 2)
        cos_sim = ops.mean(ops.sum(z*p, axis = -1)
                          ) #batchwise mean
        return -cos_sim

    def train_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        with tf.GradientTape() as tape:
            if len(self.feature_extractor.outputs) == 2:
                z1, weight1 = self.feature_extractor(img1, training = True)
                z2, weight2 = self.feature_extractor(img2, training = True)
            elif len(self.feature_extractor.outputs) == 1:
                z1 = self.feature_extractor(img1, training = True)
                z2 = self.feature_extractor(img2, training = True)
            v1, v2 = self.predictor(z1, training = True), self.predictor(z2, training = True)
            
            if self.multiview:
                v2 = ops.split(v2, n_augs, 0)
                loss = []
                for idx in range(n_augs):
                    loss.append(self.compute_loss(v1, v2[idx])
                               )
                loss = ops.mean(loss)
            else:
                loss = 0.5*(self.compute_loss(v1, v2) + self.compute_loss(v2, v1))
        trainable_params = self.feature_extractor.trainable_variables + self.predictor.trainable_variables
        grads = tape.gradient(loss, trainable_params)
        self.optimizer.apply_gradients(zip(grads, trainable_params))
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result()}
        
        if self.probe:
            with tf.GradientTape() as tape:
                class_logits = self.linear_probe(z1)
                probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
                probe_loss = ops.mean(probe_loss)
            grads = tape.gradient(probe_loss, self.linear_probe.trainable_weights)
            self.probe_optimizer.apply_gradients(zip(grads, self.linear_probe.trainable_weights))
            self.probe_loss_tracker.update_state(probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            #metrics
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
        del tape
        return output_dict
    
    def test_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        if len(self.feature_extractor.outputs) == 2:
            z1, weight1 = self.feature_extractor(img1, training = False)
            z2, weight2 = self.feature_extractor(img2, training = False)
        elif len(self.feature_extractor.outputs) == 1:
            z1 = self.feature_extractor(img1, training = False)
            z2 = self.feature_extractor(img2, training = False)
        v1, v2 = self.predictor(z1, training = False), self.predictor(z2, training = False)
        if self.multiview:
            v2 = ops.split(v2, n_augs, 0)
            loss = []
            for idx in range(n_augs):
                loss.append(self.compute_loss(v1, v2[idx])
                            )
            loss = ops.mean(loss)
        else:
            loss = 0.5*(self.compute_loss(v1, v2) + self.compute_loss(v2, v1))
        
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result()}
        
        if self.probe:
            class_logits = self.linear_probe(z1)
            probe_loss = self.probe_loss_fn(y_true = labels, y_pred = class_logits)
            probe_loss = ops.mean(probe_loss)
            self.probe_loss_tracker.update_state(probe_loss)
            output_dict.update({"probe_classification_loss" : self.probe_loss_tracker.result()})
            for comp in self.probe_metrics:
                comp.update_state(labels, class_logits)
                
            probe_metric_dict = {comp.name :  comp.result() for comp in self.probe_metrics}
            output_dict.update(probe_metric_dict)
        
        return output_dict
    
    def call(self, dataset):
        return self.test_step(dataset)

In [20]:
class SimCLR(keras.Model):
    def __init__(self, feature_extractor, embed_dims = 256, multiview = False, probe = False,
                 temperature = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.feature_extractor = feature_extractor #attentive pooling을 거친 encoder로 가정 : rep_vector, attention_Weight를 output으로
        self.embed_dims = embed_dims
        self.multiview = multiview
        self.t = temperature
        self.train_type = 'SimCLR'
        print("Train with Gradient Accumulation is recommended!")
        
        self.embed_dims = embed_dims
        self.predictor = keras.Sequential([keras.layers.Dense(units = self.embed_dims,
                                                            use_bias = False,
                                                            kernel_regularizer = keras.regularizers.l2(5e-4),
                                                            ),
                                          keras.layers.Activation("gelu"),
                                          keras.layers.BatchNormalization(),
                                          keras.layers.Dense(units = self.embed_dims),
                                          ], name = f"{self.train_type}_predictor")
        self.probe = False
        
        self.contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy(
            name="simclr_contrastive_accuracy"
        )
        self.val_contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy(
            name="simclr_contrastive_accuracy"
        )
        
        self.loss_tracker = keras.metrics.Mean(name = f"{self.train_type}_loss")
        
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
                "Multiview(>2)" : self.multiview,
               "SSL_method" : "SimSiam",
               "Linear Probe" : self.probe,
               "N_Categories" : self.probe_categories if self.probe else 0,
               "Probe Activation" : self.probe_act if self.probe else "NA"}
    def compile(self, optimizer, probe_optimizer = None, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer
        if self.probe:
            self.probe_optimizer = probe_optimizer 
    def compute_loss(self, p, z, epsilon = 1e-5, training = True):
        v1, v2 = ops.normalize(p, axis = 1, order = 2), ops.normalize(z, axis = 1, order = 2) #batch, dims
        sim_mat = ops.matmul(v1, ops.transpose(v2))/self.t #batch by batch
        batch_size = ops.shape(v1)[0]
        pseudo_labels = ops.arange(batch_size)
        if training : 
            self.contrastive_accuracy.update_state(pseudo_labels, sim_mat)
            self.contrastive_accuracy.update_state(pseudo_labels, ops.transpose(sim_mat))
        else:
            self.val_contrastive_accuracy.update_state(pseudo_labels, sim_mat)
            self.val_contrastive_accuracy.update_state(pseudo_labels, ops.transpose(sim_mat))
        loss_1 = keras.losses.sparse_categorical_crossentropy(
            pseudo_labels, sim_mat, from_logits=True
        )
        loss_2 = keras.losses.sparse_categorical_crossentropy(pseudo_labels, ops.transpose(sim_mat),
                                                             from_logits = True)
        loss = 0.5*(loss_1 + loss_2)
        
        return loss

    def train_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        with tf.GradientTape() as tape:
            if len(self.feature_extractor.outputs) == 2:
                z1, weight1 = self.feature_extractor(img1, training = True)
                z2, weight2 = self.feature_extractor(img2, training = True)
            elif len(self.feature_extractor.outputs) == 1:
                z1 = self.feature_extractor(img1, training = True)
                z2 = self.feature_extractor(img2, training = True)
            v1, v2 = self.predictor(z1, training = True), self.predictor(z2, training = True)
            
            if self.multiview:
                v2 = ops.split(v2, n_augs, 0)
                loss = []
                for idx in range(n_augs):
                    loss.append(self.compute_loss(v1, v2[idx]))
                    
                loss = ops.mean(loss)
                
            else:
                loss = self.compute_loss(v1, v2)
        trainable_params = self.feature_extractor.trainable_variables + self.predictor.trainable_variables
        grads = tape.gradient(loss, trainable_params)
        self.optimizer.apply_gradients(zip(grads, trainable_params))
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result(),
                      self.contrastive_accuracy.name : self.contrastive_accuracy.result()}
        return output_dict
    
    def test_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        if len(self.feature_extractor.outputs) == 2:
            z1, weight1 = self.feature_extractor(img1, training = True)
            z2, weight2 = self.feature_extractor(img2, training = True)
        elif len(self.feature_extractor.outputs) == 1:
            z1 = self.feature_extractor(img1, training = True)
            z2 = self.feature_extractor(img2, training = True)
        v1, v2 = self.predictor(z1, training = False), self.predictor(z2, training = False)
            
        if self.multiview:
            v2 = ops.split(v2, n_augs, 0)
            loss = []
            for idx in range(n_augs):
                loss.append(self.compute_loss(v1, v2[idx],
                                               training = False))        
            loss = ops.mean(loss)
            
        else:
            loss = self.compute_loss(v1, v2)
        
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result(),
                       self.val_contrastive_accuracy.name : self.val_contrastive_accuracy.result()
                      }
        return output_dict
    
    def call(self, dataset):
        return self.test_step(dataset)

In [21]:
class Moco(keras.Model):
    def __init__(self, feature_extractor, embed_dims, q_size = 2**13, pool_heads = 8, t = 0.07,
                 momentum_coefficient = 0.999,
                 **kwargs):
        super().__init__(**kwargs)
        self.feature_extractor = feature_extractor
        self.momentum_encoder = feature_extractor
        self.momentum_encoder.set_weights(self.feature_extractor.get_weights())
        self.m = momentum_coefficient
        self.pool_heads = pool_heads
        self.t = t
        self.q_size = q_size
        self.embed_dims = embed_dims
        self.feature_queue = keras.Variable(
            keras.utils.normalize(
                keras.random.normal(shape=(self.q_size, self.embed_dims)),
                axis=1,
                order=2,
            ),
            trainable=False, dtype = "float32"
        )
        self.projector = keras.Sequential([keras.layers.Dense(units = embed_dims),
                                           keras.layers.Activation("relu"),
                                          keras.layers.Dense(units = embed_dims, dtype = "float32")])
        self.momentum_projector = keras.models.clone_model(self.projector)
        
        self.ce_loss_fn = keras.losses.SparseCategoricalCrossentropy()
        self.loss_tracker = keras.metrics.Mean(name = "MoCo_loss")
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
               "SSL_method" : "MoCo",
               "Queue size" : self.q_size,
               "temperature" : self.t,
               "Momentum" : self.m}
    def compute_loss(self, z_a, z_b, training = True): #Info-NCE loss in the original paper
        z_b = ops.stop_gradient(z_b)
        z_a, z_b = keras.utils.normalize(z_a, axis = -1, order = 2), keras.utils.normalize(z_b, axis = -1, order = 2)
        z_a, z_b = ops.cast(z_a, "float32"), ops.cast(z_b, "float32")
        
        pos_sim_ = ops.diagonal(z_a@ops.transpose(z_b)) ; pseudolabel = ops.zeros_like(pos_sim_)
        pos_pair_similarity = ops.expand_dims(pos_sim_, axis = -1) ; del pos_sim_
        
        neg_pair_similarity = z_a@ops.cast(ops.transpose(self.feature_queue), "float32")
        logits = ops.concatenate([pos_pair_similarity, neg_pair_similarity], axis = 1)
        logits = ops.exp(logits/self.t)
        logits = logits / (ops.sum(logits, axis = -1, keepdims = True) + 1e-8)
        loss = self.ce_loss_fn(y_true = pseudolabel, y_pred = logits)
        
        if training:
            self.feature_queue.assign(
                ops.concatenate([z_a, ops.cast(self.feature_queue[:-ops.shape(z_a)[0] ],
                                              "float32")
                                ], axis=0)
            )

        return loss
    def train_step(self, dataset):
        img1, img2 = dataset
        with tf.GradientTape() as tape:
            if len(self.feature_extractor.outputs) == 2:
                z1, weight1 = self.feature_extractor(img1)
                z2, weight2 = self.momentum_encoder(img2)
            elif len(self.feature_extractor.outputs) == 1:
                z1 = self.feature_extractor(img1)
                z2 = self.momentum_encoder(img2)
            v1, v2 = self.projector(z1), self.momentum_projector(z2)
            moco_loss = self.compute_loss(v1, v2)
        grads = tape.gradient(moco_loss, 
                             self.feature_extractor.trainable_weights + self.projector.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, 
                                          self.feature_extractor.trainable_weights + self.projector.trainable_weights)
                                      )
        self.loss_tracker.update_state(moco_loss)
        
        #momentum update
        for weight, m_weight in zip(self.feature_extractor.weights, self.momentum_encoder.weights):
            m_weight.assign(self.m * m_weight + (1-self.m)*weight)
        for weight, m_weight in zip(self.projector.weights, self.momentum_projector.weights):
            m_weight.assign(self.m * m_weight + (1-self.m)*weight)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    def test_step(self, dataset):
        img1, img2 = dataset
        if len(self.feature_extractor.outputs) == 2:
            z1, weight1 = self.feature_extractor(img1)
            z2, weight2 = self.momentum_encoder(img2)
        elif len(self.feature_extractor.outputs) == 1:
            z1 = self.feature_extractor(img1)
            z2 = self.momentum_encoder(img2)
        v1, v2 = self.projector(z1), self.momentum_projector(z2)
        moco_loss = self.compute_loss(v1, v2, training = False)
        self.loss_tracker.update_state(moco_loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    def call(self, dataset):
        img1, img2 = dataset
        if len(self.feature_extractor.outputs) == 2:
            z1, weight1 = self.feature_extractor(img1)
            z2, weight2 = self.momentum_encoder(img2)
        elif len(self.feature_extractor.outputs) == 1:
            z1 = self.feature_extractor(img1)
            z2 = self.momentum_encoder(img2)
        v1, v2 = self.projector(z1), self.momentum_projector(z2)
        moco_loss = self.compute_loss(v1, v2, training = False)
        self.loss_tracker.update_state(moco_loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}

----------

# Clustering & Distillation
- SwAV, DINO

> DINO architecture:

![](https://miro.medium.com/v2/resize:fit:1400/1*huuMgEbBryxXUufW33uhvQ.png)

In [22]:
class DINO(keras.Model):
    def __init__(self, feature_extractor, embed_dims = 256, multiview = False, probe = False,
                 temperature = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.feature_extractor = feature_extractor #attentive pooling을 거친 encoder로 가정 : rep_vector, attention_Weight를 output으로
        self.embed_dims = embed_dims
        self.multiview = multiview
        self.t = temperature
        self.train_type = 'DINO'
        self.embed_dims = embed_dims
        self.predictor = keras.Sequential([keras.layers.Dense(units = self.embed_dims,
                                                            use_bias = False,
                                                            kernel_regularizer = keras.regularizers.l2(5e-4),
                                                            ),
                                          keras.layers.Activation("gelu"),
                                          keras.layers.BatchNormalization(),
                                          keras.layers.Dense(units = self.embed_dims),
                                          ], name = f"{self.train_type}_predictor")
        self.probe = False
        
        self.contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy(
            name="simclr_contrastive_accuracy"
        )
        self.val_contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy(
            name="simclr_contrastive_accuracy"
        )
        
        self.loss_tracker = keras.metrics.Mean(name = f"{self.train_type}_loss")
        
    def get_config(self):
        return {"feature_extractor_name" : self.feature_extractor.name,
               "embed_dims" : self.embed_dims,
                "Multiview(>2)" : self.multiview,
               "SSL_method" : "SimSiam",
               "Linear Probe" : self.probe,
               "N_Categories" : self.probe_categories if self.probe else 0,
               "Probe Activation" : self.probe_act if self.probe else "NA"}
    def compile(self, optimizer, probe_optimizer = None, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer
        if self.probe:
            self.probe_optimizer = probe_optimizer 
    def compute_loss(self, p, z, epsilon = 1e-5, training = True):
        v1, v2 = ops.normalize(p, axis = 1, order = 2), ops.normalize(z, axis = 1, order = 2) #batch, dims
        sim_mat = ops.matmul(v1, ops.transpose(v2))/self.t #batch by batch
        batch_size = ops.shape(v1)[0]
        pseudo_labels = ops.arange(batch_size)
        if training : 
            self.contrastive_accuracy.update_state(pseudo_labels, sim_mat)
            self.contrastive_accuracy.update_state(pseudo_labels, ops.transpose(sim_mat))
        else:
            self.val_contrastive_accuracy.update_state(pseudo_labels, sim_mat)
            self.val_contrastive_accuracy.update_state(pseudo_labels, ops.transpose(sim_mat))
        loss_1 = keras.losses.sparse_categorical_crossentropy(
            pseudo_labels, sim_mat, from_logits=True
        )
        loss_2 = keras.losses.sparse_categorical_crossentropy(pseudo_labels, ops.transpose(sim_mat),
                                                             from_logits = True)
        loss = 0.5*(loss_1 + loss_2)
        
        return loss

    def train_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        with tf.GradientTape() as tape:
            if len(self.feature_extractor.outputs) == 2:
                z1, weight1 = self.feature_extractor(img1, training = True)
                z2, weight2 = self.feature_extractor(img2, training = True)
            elif len(self.feature_extractor.outputs) == 1:
                z1 = self.feature_extractor(img1, training = True)
                z2 = self.feature_extractor(img2, training = True)
            v1, v2 = self.predictor(z1, training = True), self.predictor(z2, training = True)
            
            if self.multiview:
                v2 = ops.split(v2, n_augs, 0)
                loss = []
                for idx in range(n_augs):
                    loss.append(self.compute_loss(v1, v2[idx]))
                    
                loss = ops.mean(loss)
                
            else:
                loss = self.compute_loss(v1, v2)
        trainable_params = self.feature_extractor.trainable_variables + self.predictor.trainable_variables
        grads = tape.gradient(loss, trainable_params)
        self.optimizer.apply_gradients(zip(grads, trainable_params))
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result(),
                      self.contrastive_accuracy.name : self.contrastive_accuracy.result()}
        return output_dict
    
    def test_step(self, dataset):
        if self.probe:
            img1, img2, labels = dataset
        else:
            if self.multiview:
                img1 = dataset[0]
                img2 = dataset[1:] ; n_augs = len(img2)
                img2 = ops.concatenate(img2, axis = 0)
            else:
                img1, img2 = dataset
        
        if len(self.feature_extractor.outputs) == 2:
            z1, weight1 = self.feature_extractor(img1, training = True)
            z2, weight2 = self.feature_extractor(img2, training = True)
        elif len(self.feature_extractor.outputs) == 1:
            z1 = self.feature_extractor(img1, training = True)
            z2 = self.feature_extractor(img2, training = True)
        v1, v2 = self.predictor(z1, training = False), self.predictor(z2, training = False)
            
        if self.multiview:
            v2 = ops.split(v2, n_augs, 0)
            loss = []
            for idx in range(n_augs):
                loss.append(self.compute_loss(v1, v2[idx],
                                               training = False))        
            loss = ops.mean(loss)
            
        else:
            loss = self.compute_loss(v1, v2)
        
        self.loss_tracker.update_state(loss)
        output_dict = {self.loss_tracker.name : self.loss_tracker.result(),
                       self.val_contrastive_accuracy.name : self.val_contrastive_accuracy.result()
                      }
        return output_dict
    
    def call(self, dataset):
        return self.test_step(dataset)

----------------
# Multimodal Contrastive method
- CLIP
- SigLIP

In [23]:
class CLIP(keras.Model): #original CLIP + CLIP surgery
    def __init__(self, image_encoder, text_encoder,
                embed_dims, pool_heads = 8, t = 2.0, **kwargs):
        super().__init__(**kwargs)
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.embed_dims = embed_dims
        self.t = t
        self.mlp_image = keras.layers.Dense(units = embed_dims)
        self.mlp_text = keras.layers.Dense(units = embed_dims)
        self.pe_fn = keras_nlp.layers.RotaryEmbedding(name = "RotatoryPE")
        self.image_pooler = AttentionPooling(pool_heads, embed_dims, name = "ImageAttentionPooler")
        self.text_pooler = AttentionPooling(pool_heads, embed_dims, name = "TextAttentionPooler")
        self.loss_tracker = keras.metrics.Mean(name = "CLIP_loss")
    def get_config(self):
        return {"Image encoder name": self.image_encoder.name,
               "Text encoder name" : self.text_encoder.name,
               "embed_dims" : self.embed_dims,
               "temperature" : self.t,
               "SSL_method" : "CLIP_with_Attentional_Pooling"}
    def get_clip_loss(self, image_vector, text_vector):
        batch_size = ops.shape(image_vector)[0]
        image_vector = keras.utils.normalize(image_vector, axis = -1, order = 2)
        text_vector = keras.utils.normalize(text_vector, axis = -1, order = 2)
        
        cor_mat = tf.einsum("ab, cb->ac", image_vector, text_vector) / self.t
        cor_mat = ops.reshape(cor_mat, [batch_size, batch_size])
        pseudo_label = ops.zeros_like(cor_mat)
        diags = tf.linalg.diag_part(pseudo_label) + 1.0
        pseudo_label = tf.linalg.set_diag(pseudo_label, diags)
        pseudo_label = pseudo_label - ops.ones_like(pseudo_label)
        loss = 0.5*(ops.mean(keras.losses.CategoricalFocalCrossentropy(reduction = None, label_smoothing = 0.05)(pseudo_label, cor_mat)) + 
                    ops.mean(keras.losses.CategoricalFocalCrossentropy(reduction = None, label_smoothing = 0.05)(pseudo_label, ops.transpose(cor_mat)))
                   )
        return loss
        
    def train_step(self, dataset):
        image, text = dataset
        with tf.GradientTape() as tape: 
            image_feature, text_feature = self.image_encoder(image, training = True), self.text_encoder(text, training = True)
            batch_size, w, h, dims = ops.shape(image_feature) ; batch_size = ops.shape(image_feature)[0]
            
            image_feature = ops.reshape(image_feature, [batch_size, w*h, dims])
            image_feature = self.pe_fn(image_feature)
            text_vector = keras.layers.GlobalAveragePooling1D()(text_feature)
            if len(ops.shape(image_feature)) == 3:
                image_vector = keras.layers.GlobalAveragePooling1D()(image_feature)
            elif len(ops.shape(image_feature)) == 4:
                image_vector = keras.layers.GlobalAveragePooling2D()(image_feature)

            image_vector, text_vector = self.mlp_image(image_vector, training = True), self.mlp_text(text_vector, training = True)
            image_vector, text_vector = self.image_pooler([image_vector, image_feature], training = True)[0], self.text_pooler([text_vector, text_feature], training = True)[0]
            loss = self.get_clip_loss(image_vector, text_vector)
        
        encoder_weights = self.image_encoder.trainable_weights + self.text_encoder.trainable_weights
        mlp_weights = self.mlp_image.trainable_weights + self.mlp_text.trainable_weights
        pool_weights = self.image_pooler.trainable_weights + self.text_pooler.trainable_weights
        trainable_weights = encoder_weights + mlp_weights + pool_weights + self.pe_fn.trainable_weights
        
        grads = tape.gradient(loss, trainable_weights)
        self.optimizer.apply_gradients(zip(grads, trainable_weights))
        self.loss_tracker.update_state(loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    
    def test_step(self, dataset):
        image, text = dataset
        image_feature, text_feature = self.image_encoder(image, training = False), self.text_encoder(text, training = False)
        batch_size, w, h, dims = ops.shape(image_feature) ; batch_size = ops.shape(image_feature)[0]
            
        image_feature = ops.reshape(image_feature, [batch_size, w*h, dims])
        image_feature = self.pe_fn(image_feature)
        
        text_vector = keras.layers.GlobalAveragePooling1D()(text_feature)
        if len(ops.shape(image_feature)) == 3:
            image_vector = keras.layers.GlobalAveragePooling1D()(image_feature)
        elif len(ops.shape(image_feature)) == 4:
            image_vector = keras.layers.GlobalAveragePooling2D()(image_feature)

        image_vector, text_vector = self.mlp_image(image_vector, training = False), self.mlp_text(text_vector, training = False)
        image_vector, text_vector = self.image_pooler([image_vector, image_feature], training = False)[0], self.text_pooler([text_vector, text_feature], training = False)[0]
        loss = self.get_clip_loss(image_vector, text_vector)
        self.loss_tracker.update_state(loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    def call(self, dataset):
        return self.test_step(dataset)
    def get_full_model(self):
        inputs = self.image_encoder.inputs
        feature = self.image_encoder.output
        if len(ops.shape(feature)) == 4:
            batch_size, w, h, dims = ops.shape(feature)
            batch_size = ops.shape(feature)[0]
            feature = ops.reshape(feature, [-1, w*h, dims])
        feature = self.pe_fn(feature)
        image_vector = keras.layers.GlobalAveragePooling1D()(feature)
        z_image = self.mlp_image(image_vector)
        outputs = self.image_pooler([z_image, feature])
        return keras.Model(inputs, outputs,
                          name = f"FullModel_{self.image_encoder.name}")

In [24]:
class SigLIP(keras.Model):
    def __init__(self, image_encoder, text_encoder,
                embed_dims, pool_heads = 8, **kwargs):
        super().__init__(**kwargs)
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.pe_fn = keras_nlp.layers.RotaryEmbedding(name = "RotatoryPE")
        self.embed_dims = embed_dims
        self.mlp_image = keras.layers.Dense(units = embed_dims)
        self.mlp_text = keras.layers.Dense(units = embed_dims)
        self.image_pooler = AttentionPooling(pool_heads, embed_dims, name = "ImageAttentionPooler")
        self.text_pooler = AttentionPooling(pool_heads, embed_dims, name = "TextAttentionPooler")
        self.loss_tracker = keras.metrics.Mean(name = "SigLIP_loss")
        
        self.t = tf.Variable(1.0, trainable = True, dtype = "float32")
        self.b = tf.Variable(0.0, trainable = True, dtype = "float32")
    def get_config(self):
        return {"Image encoder name": self.image_encoder.name,
               "Text encoder name" : self.text_encoder.name,
               "embed_dims" : self.embed_dims,
               "SSL_method" : "SigLIP"}
    
    def compute_loss(self, image_vector, text_vector):
        batch_size = ops.shape(image_vector)[0]
        z_img = keras.utils.normalize(image_vector, axis = -1, order = 2)
        z_text = keras.utils.normalize(text_vector, axis = -1, order = 2)
        z_img = tf.cast(z_img, tf.float32)
        z_text = tf.cast(z_text, tf.float32)
        cor_mat = ops.dot(z_img, ops.transpose(z_text)) * ops.exp(self.t) + self.b
        cor_mat = ops.reshape(cor_mat, [batch_size, batch_size])
        
        pseudo_label = ops.zeros_like(cor_mat)
        diags = tf.linalg.diag_part(pseudo_label) + 2.0
        pseudo_label = tf.linalg.set_diag(pseudo_label, diags)
        pseudo_label = pseudo_label - ops.ones_like(pseudo_label)
        
        loss = -ops.mean(ops.log_sigmoid(ops.multiply(cor_mat, pseudo_label)))
        return loss
        
    def train_step(self, dataset):
        image, text = dataset
        with tf.GradientTape() as tape: 
            image_feature, text_feature = self.image_encoder(image, training = True), self.text_encoder(text, training = True)
            batch_size, w, h, dims = ops.shape(image_feature) ; batch_size = ops.shape(image_feature)[0]
            
            image_feature = ops.reshape(image_feature, [batch_size, w*h, dims])
            image_feature = self.pe_fn(image_feature)
            
            text_vector = keras.layers.GlobalAveragePooling1D()(text_feature)
            if len(ops.shape(image_feature)) == 3:
                image_vector = keras.layers.GlobalAveragePooling1D()(image_feature)
            elif len(ops.shape(image_feature)) == 4:
                image_vector = keras.layers.GlobalAveragePooling2D()(image_feature)

            image_vector, text_vector = self.mlp_image(image_vector, training = True), self.mlp_text(text_vector, training = True)
            image_vector, text_vector = self.image_pooler([image_vector, image_feature], training = True)[0], self.text_pooler([text_vector, text_feature], training = True)[0]
            loss = 0.5*(self.compute_loss(image_vector, text_vector) + self.compute_loss(text_vector, image_vector))
        
        encoder_weights = self.image_encoder.trainable_weights + self.text_encoder.trainable_weights
        mlp_weights = self.mlp_image.trainable_weights + self.mlp_text.trainable_weights
        pool_weights = self.image_pooler.trainable_weights + self.text_pooler.trainable_weights
        trainable_weights = encoder_weights + mlp_weights + pool_weights + [self.t, self.b] + self.pe_fn.trainable_weights
        
        grads = tape.gradient(loss, trainable_weights)
        self.optimizer.apply_gradients(zip(grads, trainable_weights))
        self.loss_tracker.update_state(loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    
    def test_step(self, dataset):
        image, text = dataset
        image_feature, text_feature = self.image_encoder(image, training = False), self.text_encoder(text, training = False)
        batch_size, w, h, dims = ops.shape(image_feature) ; batch_size = ops.shape(image_feature)[0]
            
        image_feature = ops.reshape(image_feature, [batch_size, w*h, dims])
        image_feature = self.pe_fn(image_feature)
        text_vector = keras.layers.GlobalAveragePooling1D()(text_feature)
        if len(ops.shape(image_feature)) == 3:
            image_vector = keras.layers.GlobalAveragePooling1D()(image_feature)
        elif len(ops.shape(image_feature)) == 4:
            image_vector = keras.layers.GlobalAveragePooling2D()(image_feature)

        image_vector, text_vector = self.mlp_image(image_vector, training = False), self.mlp_text(text_vector, training = False)
        image_vector, text_vector = self.image_pooler([image_vector, image_feature], training = False)[0], self.text_pooler([text_vector, text_feature], training = False)[0]
        loss = 0.5*(self.compute_loss(image_vector, text_vector) + self.compute_loss(text_vector, image_vector))
        self.loss_tracker.update_state(loss)
        return {self.loss_tracker.name : self.loss_tracker.result()}
    def call(self, dataset):
        return self.test_step(dataset)
    def get_full_model(self):
        inputs = self.image_encoder.inputs
        feature = self.image_encoder.output
        if len(ops.shape(feature)) == 4:
            batch_size, w, h, dims = ops.shape(feature)
            batch_size = ops.shape(feature)[0]
            feature = ops.reshape(feature, [-1, w*h, dims])
        feature = self.pe_fn(feature)
        
        image_vector = keras.layers.GlobalAveragePooling1D()(feature)
        z_image = self.mlp_image(image_vector)
        outputs = self.image_pooler([z_image, feature])
        return keras.Model(inputs, outputs,
                          name = f"FullModel_{self.image_encoder.name}")

In [25]:
class SPARC(keras.Model): 
    #Reference : https://arxiv.org/abs/2401.09865 "Improving fine-grained understanding in image-text pre-training"
    def __init__(self, image_encoder, text_encoder,
                embed_dims, pool_heads = 8, preprocessor = None,
                 **kwargs):
        super().__init__(**kwargs)
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.preprocessor = preprocessor
        self.pe_fn = keras_nlp.layers.RotaryEmbedding(name = "RotatoryPE")
        self.embed_dims = embed_dims
        self.mlp_image = keras.layers.Dense(units = embed_dims)
        self.mlp_text = keras.layers.Dense(units = embed_dims)
        self.image_pooler = AttentionPooling(pool_heads, embed_dims, name = "ImageAttentionPooler")
        self.text_pooler = AttentionPooling(pool_heads, embed_dims, name = "TextAttentionPooler")
        self.ce_fn = keras.losses.CategoricalCrossentropy(from_logits=True, reduction = None)
        self.loss_tracker = keras.metrics.Mean(name = "SPARC_loss")
        self.global_loss_tracker = keras.metrics.Mean(name = "SPARC_global_loss")
        self.local_loss_tracker = keras.metrics.Mean(name = "SPARC_local_loss")
        
        self.t = tf.Variable(1.0, trainable = True, dtype = "float32")
        self.b = tf.Variable(0.0, trainable = True, dtype = "float32")
        self.threshold = tf.Variable(0.5, trainable = True, dtype = "float32")
        
    def get_config(self):
        return {"Image encoder name": self.image_encoder.name,
               "Text encoder name" : self.text_encoder.name,
               "embed_dims" : self.embed_dims,
               "SSL_method" : "SPARC"}
    
    def compute_global_loss(self, image_vector, text_vector): #<- SigLIP
        batch_size = ops.shape(image_vector)[0]
        z_img = keras.utils.normalize(image_vector, axis = -1, order = 2)
        z_text = keras.utils.normalize(text_vector, axis = -1, order = 2)
        z_img = tf.cast(z_img, tf.float32)
        z_text = tf.cast(z_text, tf.float32)
        cor_mat = ops.dot(z_img, ops.transpose(z_text)) * ops.exp(self.t) + self.b
        cor_mat = ops.reshape(cor_mat, [batch_size, batch_size])
        
        pseudo_label = ops.zeros_like(cor_mat)
        diags = tf.linalg.diag_part(pseudo_label) + 2.0
        pseudo_label = tf.linalg.set_diag(pseudo_label, diags)
        pseudo_label = pseudo_label - ops.ones_like(pseudo_label)
        
        loss = -ops.mean(ops.log_sigmoid(ops.multiply(cor_mat, pseudo_label)))
        loss = tf.cast(loss, tf.float32)
        return loss
    
    def text_compare(self, text_label, text_logit, mask = None):
        loss = self.ce_fn(text_label, text_logit)
        loss = tf.cast(loss, tf.float32)
        if mask == None:
            return ops.mean(loss)
        else:
            loss *= tf.cast(mask, tf.float32)
            loss = (ops.sum(loss))/(ops.sum(tf.cast(mask, tf.float32)) + 1e-4)
            return loss
        
    def compute_loss(self, image_sequence, text_sequence, mask = None):
        batch_size = ops.shape(image_sequence)[0]
        _, token_len, dim_ = ops.shape(text_sequence)
        if len(ops.shape(image_sequence)) == 3:
            _, image_len, dim_ = ops.shape(image_sequence)
        elif len(ops.shape(image_sequence)) == 4:
            _, w, h, dim_ = ops.shape(image_sequence)
            image_sequence = ops.reshape(image_sequence, [batch_size, w*h, dim_])
            image_len = w*h
            
        image_sequence, text_sequence = self.mlp_image(image_sequence), self.mlp_text(text_sequence) 
        image_sequence = self.pe_fn(image_sequence)
        # [batch, image_len, embed_dims] / [batch, token_len, embed_dims], respectively.
        
        z_image, att_weight = self.image_pooler([keras.layers.GlobalAveragePooling1D()(image_sequence),
                                                     image_sequence])
        z_text, att_weight_ = self.text_pooler([keras.layers.GlobalAveragePooling1D()(text_sequence),
                                                     text_sequence])
        #1. global alignment loss
        global_loss = 0.5*(self.compute_global_loss(z_image, z_text) + self.compute_global_loss(z_text, z_image))
        #2. Fine Grained local loss
        
        #a. get similarity matrix b/w text sequence and image sequence
        sim_matrix = ops.einsum("atd, aid -> ati", text_sequence, image_sequence) #batch, token_len, image_len
        sim_matrix = (sim_matrix - ops.min(sim_matrix, axis = -1, keepdims = True)) / (1e-4 + ops.max(sim_matrix, axis = -1, keepdims = True) - ops.min(sim_matrix, axis = -1, keepdims = True))
        sim_matrix = tf.cast(sim_matrix, tf.float32)
        
        sim_matrix = ops.clip(sim_matrix, self.threshold, 1e4)
        attended_text_sequence = ops.einsum("ati, aid -> atd", sim_matrix, image_sequence)
        
        text_logit = ops.einsum("atd, aqd -> atq", 
                                keras.utils.normalize(text_sequence, axis = -1, order = 2), 
                                keras.utils.normalize(attended_text_sequence, axis = -1, order = 2))/self.t
        
        pseudo_text_label = ops.tile(ops.expand_dims(ops.eye(token_len), axis = 0),
                                     [batch_size,1,1])
        local_loss = self.text_compare(pseudo_text_label, text_logit, mask)
        loss = 0.5*global_loss + 1.0*local_loss
        return loss, global_loss, local_loss
        
    def train_step(self, dataset):
        image, text = dataset
        if self.preprocessor != None:
            text_mask = self.preprocessor(text)["padding_mask"]
            text_mask = tf.cast(text_mask, dtype = tf.int32)
        else:
            text_mask = None
        with tf.GradientTape() as tape: 
            image_feature, text_feature = self.image_encoder(image, training = True), self.text_encoder(text, training = True)
            loss, global_loss, local_loss = self.compute_loss(image_feature, text_feature, text_mask)
        
        encoder_weights = self.image_encoder.trainable_weights + self.text_encoder.trainable_weights
        mlp_weights = self.mlp_image.trainable_weights + self.mlp_text.trainable_weights
        pool_weights = self.image_pooler.trainable_weights + self.text_pooler.trainable_weights
        trainable_weights = encoder_weights + mlp_weights + pool_weights + [self.t, self.b] + self.pe_fn.trainable_weights
        
        grads = tape.gradient(loss, trainable_weights)
        self.optimizer.apply_gradients(zip(grads, trainable_weights))
        self.loss_tracker.update_state(loss)
        self.global_loss_tracker.update_state(global_loss)
        self.local_loss_tracker.update_state(local_loss)
        return {self.loss_tracker.name : self.loss_tracker.result(),
               self.global_loss_tracker.name : self.global_loss_tracker.result(),
               self.local_loss_tracker.name : self.local_loss_tracker.result()
               }
    
    def test_step(self, dataset):
        image, text = dataset
        if self.preprocessor != None:
            text_mask = self.preprocessor(text)["padding_mask"]
            text_mask = tf.cast(text_mask, tf.int32)
        else:
            text_mask = None
            
        image_feature, text_feature = self.image_encoder(image, training = False), self.text_encoder(text, training = False)
        loss, global_loss, local_loss = self.compute_loss(image_feature, text_feature, text_mask)
        
        self.loss_tracker.update_state(loss)
        self.global_loss_tracker.update_state(global_loss)
        self.local_loss_tracker.update_state(local_loss)
        return {self.loss_tracker.name : self.loss_tracker.result(),
               self.global_loss_tracker.name : self.global_loss_tracker.result(),
               self.local_loss_tracker.name : self.local_loss_tracker.result()
               }
    
    def call(self, dataset):
        return self.test_step(dataset)
    def get_full_model(self):
        inputs = self.image_encoder.inputs
        feature = self.image_encoder.output
        feature = self.mlp_image(feature)
        if len(ops.shape(feature)) == 4:
            batch_size, w, h, dims = ops.shape(feature)
            batch_size = ops.shape(feature)[0]
            feature = ops.reshape(feature, [-1, w*h, dims])
        feature = self.pe_fn(feature)
        
        image_vector = keras.layers.GlobalAveragePooling1D()(feature)
        outputs = self.image_pooler([image_vector, feature])
        return keras.Model(inputs, outputs,
                          name = f"FullModel_{self.image_encoder.name}")

# Visualization

In [26]:
def att_visualize(model, images, res, thresholding = False):
    # Use jet colormap to colorize heatmap
    try:
        model = model.get_full_model()
    except Exception as e:
        print("Using Raw model with Att pooling","\n","Possible error:","\n",e)
        pass
    try:
        att_weights = model(images)[-1] 
        att_weights_ = att_weights[:, :, 0, :] #batch, heads, cls_token, w*h
        _, heads, token_length = ops.shape(att_weights_) ; batch_size = ops.shape(att_weights_)[0]
        token_length = tf.cast(token_length, tf.float32)

        w = tf.cast(tf.math.sqrt(token_length), tf.int32)
        heatmap = tf.reshape(att_weights_, [batch_size, heads, w, w])
        M = tf.reduce_max(heatmap, axis = [2, 3], keepdims = True)
        m = tf.reduce_min(heatmap, axis = [2, 3], keepdims = True)
        heatmap = (heatmap - m) / (M-m + 1e-4)
        if thresholding:
            threshold = ops.mean(heatmap, [2,3], keepdims = True)
            heatmap = ops.where(heatmap < threshold, 0.0, heatmap)
        else:
            pass
        heatmap = ops.reshape(heatmap, [-1, w, w])
        heatmap = tf.map_fn(fn = lambda x : cv2.resize(np.array(x).astype("float32"), [res, res]),
                           elems = heatmap)
        heatmap = ops.reshape(heatmap, [batch_size, heads, res, res, 1])
        imposed = ops.multiply(images[:, tf.newaxis, ...], heatmap)
        imposed = tf.cast(imposed, tf.uint8)    
        return imposed
    except Exception as e:
        print("Error raised:", e)
        pass

# Test - drive

In [27]:
test_drive = False

In [28]:
if test_drive:
    res = 256
    batch_size = 8
    gc_configs = get_gcvit_configs(res, 64, "GC_ViT_xxtiny")
    gc_configs["level_depth"] = [1,1,2,2]
    print(gc_configs)
    dataset = tfds.load("beans", split = 'test')
    def map_fn(dataset):
        image = tf.image.resize_with_pad(dataset["image"], res, res, antialias = True)
        image = tf.cast(image, tf.uint8)
        return image
    train_ds = dataset.map(map_fn).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    ssl_fn = get_map_fn(res, "image", "ssl", 4)
    train_ds = train_ds.unbatch().map(ssl_fn).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    for imgs in train_ds.take(1):
        sets = imgs
    for index in range(batch_size):
        fig, axes = plt.subplots(2,2, figsize = (7,7))
        axes = axes.flatten()
        for i, ax in enumerate(axes):
            ax.imshow(ops.cast(sets[i][index], "uint8"))
            ax.set_title(f"In-Batch index : {index}")
        print("=================================================")
        plt.show()
    

In [29]:
if test_drive:
    
    vit_model = get_full_model("effnet", res = res, pe_type = None, att_depth = 1)
    vit_model.summary()
    
    barlow = SimCLR(vit_model, embed_dims = 768, 
                  multiview = True
                 )
    barlow.compile(optimizer = keras.optimizers.Adam())
    print(barlow(sets))
    print(f"GFlops : {get_flops(vit_model, imgs[:1])}")
    result = barlow.train_step(sets)
    print(result)
    
    imgs = att_visualize(barlow.feature_extractor, sets[0], res = res)
    fig, axes = plt.subplots(8, 8, figsize = (25,25))
    for row in range(8):
        for col in range(8):
            axes[row][col].imshow(imgs[row, col, ...])
    plt.show()